In [2]:
from datetime import timedelta, date
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import *
import time


#star rating buckets
def srch_region_star_rating(star_rating):
    if star_rating == '2.0':
        return '2'
    elif star_rating == '2.5':
        return '2.5'
    elif star_rating == '3.0':
        return '3'
    elif star_rating == '3.5':
        return '3.5'
    elif star_rating == '4.0':
        return '4'
    elif star_rating == '1.0' or star_rating == '1.5':
        return '1-1.5'
    elif star_rating == '4.5' or star_rating == '5.0':
        return '4.5-5'
    else:
        return '0'

#udf for creating city buckets
def srch_region_city(city,model_name):
	if model_name == 'CANADA':
		if city == 'TORONTO':
			return 'TORONTO'
		elif city == 'NIAGARA FALLS':
			return 'NIAGARA FALLS'
		elif city == 'MONTREAL':
			return 'MONTREAL'
		elif city == 'VANCOUVER':
			return 'VANCOUVER'
		elif city == 'QUEBEC':
			return 'QUEBEC'
		elif city == 'BANFF':
			return 'BANFF'
		elif city == 'OTTAWA':
			return 'OTTAWA'
		elif city == 'EDMONTON':
			return 'EDMONTON'
		elif city == 'VICTORIA':
			return 'VICTORIA'
		elif city == 'CALGARY':
			return 'CALGARY'
		elif city == 'WHISTLER':
			return 'WHISTLER'
		elif city in ('NIAGARA-ON-THE-LAKE','CANMORE','JASPER','KELOWNA','HALIFAX','MISSISSAUGA','LAKELOUISE','COLLINGWOOD','VERNON','PENTICTON','GATINEAU','DORVAL','MARKHAM'):
			return'G12'
		elif city in ('MONT-TREMBLANT','TOFINO','BLUEMOUNTAINS','OSOYOOS','UCLUELET','SAINT-SAUVEUR','WESTKELOWNA','LAMALBAIE','SAINTE-ADELE'):
			return'G13'
		elif city in ('WINNIPEG','LONDON','RICHMOND','BARRIE','REVELSTOKE','VAUGHAN','ORILLIA','BURNABY','SURREY'):
			return 'G14'
		else:
			return 'others'
	else:
		return 'NA'

#udf for hotel type
def srch_region_Hotel_type(Hotel_type,model_name):
	if model_name == 'CANADA':
		if Hotel_type == 'HOTEL':
			return 'HOTEL'
		elif Hotel_type == 'HOTEL RESORT':
			return 'HOTEL RESORT'
		elif Hotel_type == 'MOTEL':
			return 'MOTEL'
		elif Hotel_type == 'CONDOMINIUM RESORT' or Hotel_type == 'CONDO':
			return 'CONDO'
		elif Hotel_type in ('LODGE' , 'BED & BREAKFAST' , 'HOSTEL/BACKPACKER ACCOMMODATION' , 'CABIN' , 'GUEST HOUSE'):
			return 'Budget_hotel'
		elif Hotel_type in ('APART-HOTEL', 'INN'):
			return 'Family_Premium'
		else:
			return 'Others'
	
	elif model_name == 'US':
		if Hotel_type == 'HOTEL':
			return 'HOTEL'
		elif Hotel_type == 'HOTEL RESORT':
			return 'HOTEL RESORT'
		elif Hotel_type == 'MOTEL':
			return 'MOTEL'
		elif Hotel_type in ('CONDOMINIUM RESORT' ,'CONDO'):
			return 'CONDO'
		elif Hotel_type in ('LODGE' , 'BED & BREAKFAST' , 'HOSTEL/BACKPACKER ACCOMMODATION' , 'CABIN' , 'GUEST HOUSE'):
			return 'Budget_hotel'
		elif Hotel_type in ('APART-HOTEL' ,  'INN' , 'APARTMENT', 'VILLA', 'PRIVATE VACATION HOME'):
			return 'Family_Premium'
		else:
			return 'others'
		
	elif model_name == 'Mexico':
		if Hotel_type == 'HOTEL':
			return 'HOTEL'
		elif Hotel_type == 'HOTEL RESORT':
			return 'HOTEL RESORT'
		elif Hotel_type == 'ALL-INCLUSIVE':
			return 'ALL-INCLUSIVE'
		elif Hotel_type in ('CONDOMINIUM RESORT' ,'CONDO'):
			return 'CONDO'
		elif Hotel_type in ('LODGE' , 'BED & BREAKFAST' , 'HOSTEL/BACKPACKER ACCOMMODATION'):
			return 'Budget_hotel'
		elif Hotel_type in ('APART-HOTEL' ,'INN' , 'VILLA' , 'APARTMENT'):
			return 'Family_Premium'
		else:
			return 'others'
		
	elif model_name=="CARIBBEAN":
		if Hotel_type == 'HOTEL':
			return 'HOTEL'
		elif Hotel_type == 'HOTEL RESORT':
			return 'HOTEL RESORT'
		elif Hotel_type == 'ALL-INCLUSIVE':
			return 'ALL-INCLUSIVE'
		elif Hotel_type in ('CONDOMINIUM RESORT' ,'CONDO'):
			return 'CONDO'
		elif Hotel_type in ('APART-HOTEL' ,'INN' , 'VILLA' , 'APARTMENT'):
			return 'Family_Premium'
		else:
			return 'others'
		
	elif model_name == 'Other Region':
		if Hotel_type == 'HOTEL':
			return 'HOTEL'
		elif Hotel_type == 'HOTEL RESORT':
			return 'HOTEL RESORT'
		elif Hotel_type == 'APART-HOTEL':
			return 'APART-HOTEL'
		elif Hotel_type in ('GUEST HOUSE' ,'BED & BREAKFAST', 'HOSTEL/BACKPACKER ACCOMMODATION', 'MOTEL' , 'HOSTAL (BUDGET HOTEL)'):
			return 'Budget_hotel'
		elif Hotel_type in ('APARTMENT','RESIDENCE','TOWNHOUSE','INN'):
			return 'Budget_hotel'
		else:
			return 'others'
	
	elif model_name=="North America":
		if Hotel_type == 'HOTEL':
			return 'HOTEL'
		elif Hotel_type == 'MOTEL':
			return 'MOTEL'
		elif Hotel_type in ('LODGE' ,  'BED & BREAKFAST' ,  'COTTAGE' ,  'GUEST HOUSE' ,  'CABIN'):
			return 'Budget_hotel'
		elif Hotel_type in ( 'APARTMENT' ,  'HOTEL RESORT' ,  'CONDO' ,  'INN' ,  'APART-HOTEL' ,  'CARAVAN PARK'):
			return 'Family_Premium'
		else:
			'others'
	else:
		return 'NOT APPLICABLE'

##udf for market bucket
def srch_region_market(market,model_name):
	if model_name == 'CANADA':
		if market == 'TORONTO, ON, CAN':
			return 'TORONTO, ON, CAN'
		elif market == 'NIAGARA FALLS, ON, CAN':
			return 'NIAGARA FALLS, ON, CAN'
		elif market == 'MONTREAL, QC, CAN':
			return 'MONTREAL, QC, CAN'
		elif market == 'BANFF AREA, AB, CAN':
			return 'BANFF AREA, AB, CAN'
		elif market == 'VANCOUVER, BC, CAN':
			return 'VANCOUVER, BC, CAN'
		elif market == 'QUEBEC CITY, QC, CAN':
			return 'QUEBEC CITY, QC, CAN'
		elif market == 'EDMONTON, AB, CAN':
			return 'EDMONTON, AB, CAN'
		elif market == 'OTTAWA, ON, CAN':
			return 'OTTAWA, ON, CAN'
		elif market == 'OKANAGAN VALLEY, BC, CAN':
			return 'OKANAGAN VALLEY, BC, CAN'
		elif market in ('VICTORIA, BC, CAN' ,'CALGARY, AB, CAN' ,'JASPER AREA, AB, CAN'):
			return 'G10'
		elif market in ('WHISTLER, BC, CAN' ,'NIAGARA-ON-THE-LAKE, ON, CAN' ,'MONT TREMBLANT, QC, CAN' , 'UCLUELET, BC, CAN'):
			return 'G11'
		elif market in ('COLLINGWOOD, ON, CAN' , 'BC INTERIOR SKI, BC, CAN' ,'HALIFAX, NS, CAN','WINNIPEG, MB, CAN' ,'LONDON, ON, CAN' ,'KAMLOOPS, BC, CAN'):
			return 'G12'
		else:
			return 'others'
	
	elif model_name == 'US':
		return 'NA'
		
	elif model_name == 'Mexico':
		if market == 'MEXICO - RIVIERA MAYA, PLAYA DEL CARMEN & TULUM':
			return 'MEXICO - RIVIERA MAYA, PLAYA DEL CARMEN & TULUM'
		elif market == 'MEXICO - CANCUN & ISLA MUJERES':
			return 'MEXICO - CANCUN & ISLA MUJERES'
		elif market == 'MEXICO - PUERTO VALLARTA, NUEVO VALLARTA & RIVIERA NAYARIT':
			return 'MEXICO - PUERTO VALLARTA, NUEVO VALLARTA & RIVIERA NAYARIT'
		elif market == 'MEXICO - LOS CABOS':
			return 'MEXICO - LOS CABOS'
		elif market == 'CENTRAL AMERICA - COSTA RICA - GUANACASTE':
			return 'CENTRAL AMERICA - COSTA RICA - GUANACASTE'
		elif market == 'CENTRAL AMERICA - BELIZE':
			return 'CENTRAL AMERICA - BELIZE'
		elif market == 'CENTRAL AMERICA - COSTA RICA - PUNTARENAS':
			return 'CENTRAL AMERICA - COSTA RICA - PUNTARENAS'
		elif market == 'CENTRAL AMERICA - COSTA RICA - OTHER':
			return 'CENTRAL AMERICA - COSTA RICA - OTHER'
		elif market == 'MEXICO - COZUMEL':
			return 'MEXICO - COZUMEL'
		elif market == 'MEXICO - HUATULCO':
			return 'MEXICO - HUATULCO'
		elif market == 'MEXICO - IXTAPA & ZIHUATANEJO':
			return 'MEXICO - IXTAPA & ZIHUATANEJO'
		elif market == 'CENTRAL AMERICA - HONDURAS':
			return 'CENTRAL AMERICA - HONDURAS'
		elif market == 'CENTRAL AMERICA - PANAMA - OTHER':
			return 'CENTRAL AMERICA - PANAMA - OTHER'
		elif market in ( 'CENTRAL AMERICA - COSTA RICA - SAN JOSE' , 'CENTRAL AMERICA - PANAMA - PANAMA CITY' ,'MEXICO - MEXICO CITY' ,'MEXICO - MEXICO CITY'\
						,'CENTRAL AMERICA - NICARAGUA','MEXICO - MAZATLAN','MEXICO - MANZANILLO','CENTRAL AMERICA - EL SALVADOR','MEXICO - ACAPULCO'):
			return 'hclt'
		else:
			return 'others'
		
	elif model_name=="CARIBBEAN":
		if market == 'PUNTA CANA, DOMINICAN REPUBLIC':
			return 'PUNTA CANA, DOMINICAN REPUBLIC'
		elif market == 'TURKS AND CAICOS':
			return 'TURKS AND CAICOS'
		elif market == 'MONTEGO BAY, JAMAICA':
			return 'MONTEGO BAY, JAMAICA'
		elif market == 'ST. LUCIA':
			return 'ST. LUCIA'
		elif market == 'OCHO RIOS, JAMAICA':
			return 'OCHO RIOS, JAMAICA'
		elif market == 'NEGRIL, JAMAICA':
			return 'NEGRIL, JAMAICA'
		elif market == 'BARBADOS':
			return 'BARBADOS'
		elif market in ('ARUBA' , 'PUERTO PLATA, DOMINICAN REPUBLIC' , 'ANTIGUA AND BARBUDA'):
			return 'G8'
		elif market in ('NASSAU, BAHAMAS', 'DOMINICAN REPUBLIC ALL OTHER' , 'BERMUDA','BAHAMAS, OUT ISLANDS'):
			return 'G9'
		elif market in ('SAN JUAN, PUERTO RICO' ,'PARADISE ISLAND, BAHAMAS' ,'CAYMAN ISLANDS','SINT MAARTEN, DUTCH' ,'CURACAO','ST. MARTIN, FRENCH'):
			return 'G10'
		else:
			return 'others'
		
	elif model_name == 'Other Region':
		return 'NA'
	
	elif model_name=="North America":
		if  market in ('GRAVENHURST - BRACEBRIDGE - HUNTSVILLE, ON, CAN' ,  'ONTARIO WEST, ON, CAN' ,  'ABBOTSFORD - CHILLIWACK, BC, CAN' \
						, 'MAINE - COAST' ,  'ONTARIO EAST, ON, CAN' ,'VANCOUVER ISLAND, BC, CAN' ,  'BURLINGTON, VT' ,'CHARLOTTETOWN, PEI, CAN'): 
			return 'G1'
		elif market in ( 'ALBERTA EAST, AB, CAN' ,  'REGINA, SK, CAN' ,  'KINGSTON, ON, CAN' , 'SASKATOON, SK, CAN' ,'MONCTON, NB, CAN' \
						,'TRI-CITIES (KW-GUELPH-CAMBRIDGE), ON, CAN'): 
			return 'G2'
		elif market in ('WINDSOR, ON, CAN' ,  'RED DEER, AB, CAN' ,  'BRITISH COLUMBIA SOUTH, BC, CAN' ,  'GRAND FORKS, ND' ,  'SUDBURY, ON, CAN' , \
						'ONTARIO NORTH, ON, CAN','BANGOR, ME' ,  'HAMILTON - BRANTFORD, ON, CAN' ,  'BELLEVILLE - TRENTON - COBOURG, ON, CAN' ,'FARGO, ND'): 
			return 'G3'
		elif market in ('LAKE PLACID, NY' ,  'PARKSVILLE, VANCOUVER ISLAND, BC, CAN' ,  'LAKE GEORGE, NY','OGUNQUIT, ME' , 'EVERETT, WA' , \
						'COASTAL, BC, CAN' ,  'GROVE CITY, PA','BELLINGHAM, WA' ,  'NORTH CONWAY, NH' ,  'NEWFOUNDLAND OTHER, NL, CAN' \
						,  'NOVA SCOTIA SOUTH, NS, CAN' ,  'PORTSMOUTH, NH'): 
			return 'G4'
		elif market in ('LETHBRIDGE, AB, CAN' ,  'FREDERICTON, NB, CAN' ,  'BRANDON, MB, CAN',  'ERIE, PA' ,'NANAIMO, VANCOUVER ISLAND, BC, CAN' ,\
						'SASKATCHEWAN SOUTH, SK, CAN' ,  'PRINCE GEORGE - QUESNEL, BC, CAN' ,  'MEDICINE HAT, AB, CAN' ,  'GRANDE PRAIRIE, AB, CAN' \
						,  'ALBERTA NORTH, AB, CAN' ,  'DURHAM, ON, CAN' ,  'BRITISH COLUMBIA CENTRAL, BC, CAN' , 'ONTARIO SOUTHWEST, ON, CAN' ,\
						'SAULT STE. MARIE, ON, CAN' ,  'THUNDER BAY, ON, CAN' ,  'SAINT JOHN, NB, CAN' ,  'GREAT FALLS, MT' ,  'MINOT, ND (AREA)' \
						,  'BRITISH COLUMBIA NORTH, BC, CAN' ,  'NORTH BAY, ON, CAN' ,  'PETERBOROUGH, ON, CAN' \
						,  'CAPE BRETON ISLAND, NS, CAN' ,  'ALBERTA WEST, AB, CAN' ,  'HUDSON VALLEY - POUGHKEEPSIE, NY' \
						,  'FORT MCMURRAY, AB, CAN' ,  'NOVA SCOTIA NORTH, NS, CAN' ,  'WATERTOWN, NY' \
						,  'EDMUNSTON, NB, CAN' ,  'OLYMPIC NATIONAL PARK AREA, WA' ,  'CEDAR CITY - BRYCE CANYON NATIONAL PARK, UT' \
						,  'SUMMERSIDE, PEI, CAN' ,  'SALEM, OR (AREA)' ,  'BUTTE - HELENA, MT' ,  'BECKLEY, WV'): 
			return 'G5'
		elif market in ( 'SAGINAW, MI' ,  'WHITE MOUNTAINS, NH' ,  'FINGER LAKES NEW YORK' ,  'NEW YORK - WEST' , 'NEW BRUNSWICK NORTH, NB, CAN'  \
						,  'SASKATCHEWAN NORTH, SK, CAN' ,  'MANITOBA SOUTH, MB, CAN' ,  'BAR HARBOR, ME' \
						,  'WASHINGTON NORTHWEST' ,  'VERMONT CENTRAL' ,  'ALBERTA SOUTH, AB, CAN' \
						,  'WENATCHEE-LEAVENWORTH,WA' ,  'GANANOQUE, ON, CAN' ,  'PLATTSBURGH, (NY)' \
						,  'STRAITS OF MACKINAC, MI' ,  'SANDUSKY, OH' ,  'MANCHESTER, NH' \
						,  'ITHACA, NY' ,  'MINNESOTA NORTHWEST' ,  'MISSOULA, MT (AREA)' \
						,  'YELLOWKNIFE, NT, CAN' ,  'PEI OTHER, PEI, CAN' ,  'BILLINGS, MT (AREA)' \
						,  'RAPID CITY - MOUNT RUSHMORE, SD' ,  'ANDOVER, MA', 'YUMA, AZ' ,  'PORT HURON, MI' \
						,  'LANCASTER, PA' ,  'FORT LEE - PARAMUS, NJ' ,  'LEXINGTON, KY' \
						,  'MANITOBA NORTH, MB, CAN' ,  'ABERDEEN - OCEAN SHORES, WA' ,  'CANADA TERRITORIES, YT, CAN' \
						,  'KNOXVILLE, TN' ,  'BURLINGTON - MOUNT VERNON, WA' ,  'FREDERICKSBURG, VA' \
						,  'ARIZONA SOUTHEAST' ,  'WINDHAM COUNTY, VT' ,  'KENAI PENINSULA, AK' \
						,  'COLUMBIA, SC' ,  'CALIFORNIA DESERTS SOUTH'): 
			return 'G6'
		elif  market in ('ROME-UTICA, NY' ,  'SANDPOINT, ID (AREA)' ,  'MOAB - GREEN RIVER, UT' ,  'WHITEHORSE, YT, CAN' , 'NEW YORK - NORTHEAST'  \
							,  'SARNIA, ON, CAN' ,  'AUGUSTA, ME' ,  'LAKE POWELL - GLEN CANYON NATIONAL PARK, AZ' \
							,  'NEW YORK - SOUTH' ,  'BISMARK, ND' ,  'ALABAMA NORTH' \
							,  'ASHLAND - MEDFORD, OR' ,  'WASHINGTON NORTHEAST' ,  'THE BERKSHIRES, MA' \
							,  'EUGENE, OR (AREA)' ,  'GRAND RAPIDS, MI' ,  'WASHINGTON SOUTHWEST' \
							,  'LAKE HAVASU CITY, AZ' ,  'SARATOGA, NY' ,  'WYTHEVILLE - VA' \
							,  'ARCATA - EUREKA, CA' ,  'SOUTH CAROLINA EAST' ,  'CHATTANOOGA, TN' \
							,  'TOLEDO, OH' ,  'IDAHO FALLS, ID' ,  'WEST VIRGINIA SOUTH' \
							,  'SPRINGFIELD, MA' ,  'OLYMPIA, WA' ,  'GETTYSBURG - PA'): 
			return 'G7'
		else:return 'Others'
	else:return 'NOT APPLICABLE'		
		
def srch_model_name(region):
	if region == 'CANADA':
		return region
		
	elif region in   ('FLORIDA (USA)','NEW YORK CITY','CALIFORNIA','GAMING (USA)','NORTHEAST (USA)','MOUNTAIN (USA)','MIDWEST (USA)','CENTRAL (USA)','SF PACNW','ORLANDO','HAWAII'):
		return 'US'
		
	elif region in ("MEXICO - DOMESTIC REGION",'MEXICO - RESORTS REGION','CENTRAL AMERICA REGION'):
		return 'Mexico'
		
	elif region=="CARIBBEAN":
		return 'CARIBBEAN'
		
	elif region in ('SPAIN & PORTUGAL' , 'ROME & ITALY RESORTS', 'MID-ATLANTIC' , 'GREECE & TURKEY', 'PARIS' , 'OCEANIA' ,'ITALY NORTH' , 'LONDON, ENGLAND', 'TAIWAN' \
					,'THAILAND' , 'MIDDLE EAST AND INDIAN OCEAN' ,'EUROPE REGIONAL TERRITORIES' , 'UK & IRELAND' ,'AFRICA AND EAST MED' , 'EASTERN EUROPE' \
					,'GERMANY, AUSTRIA & SWITZERLAND' , 'JAPAN & MICRONESIA' ,'PHILIPPINES' , 'BENELUX' , 'FRANCE' ,'INDONESIA' , 'INDIAN SUBCONTINENT'  ,'SOUTH AMERICA'\
					'HONG KONG & MACAU','SOUTHEAST ASIA EMERGING' , 'NORDIC' ,'MAINLAND CHINA' , 'SOUTH KOREA', 'SINGAPORE (REGION)','MALAYSIA', 'UNKNOWN'):
		return 'Other Region'
	
	elif region=="NORTH AMERICA REGIONAL TERRITORIES":
		return 'North America'
		
	else :
		return "NOT APPLICABLE"

def srch_region_bucket(region):
	if region == 'Other Region':
		if region == 'SPAIN & PORTUGAL':
			return 'SPAIN & PORTUGAL'
		elif region == 'ROME & ITALY RESORTS':
			return 'ROME & ITALY RESORTS'
		elif region == 'MID-ATLANTIC':
			return 'MID-ATLANTIC'
		elif region == 'GREECE & TURKEY':
			return 'GREECE & TURKEY'
		elif region == 'OCEANIA':
			return 'OCEANIA'
		elif region == 'ITALY NORTH':
			return 'ITALY NORTH'
		elif region == 'THAILAND':
			return 'THAILAND'
		elif region == 'MIDDLE EAST AND INDIAN OCEAN':
			return 'MIDDLE EAST AND INDIAN OCEAN'
		elif region == 'PARIS' or region == 'LONDON' or region == 'ENGLAND':
			return 'G9'
		elif region == 'EUROPE REGIONAL TERRITORIES' or region == 'UK & IRELAND':
			return 'G10'
		elif region == 'SOUTH AMERICA' or region == 'AFRICA AND EAST MED':
			return 'G11'
		elif region == 'EASTERN EUROPE' or region == 'GERMANY, AUSTRIA & SWITZERLAND' or region == 'BENELUX' or region == 'FRANCE' or region == 'NORDIC':
			return 'G12'
		elif region == 'JAPAN & MICRONESIA' or region == 'HONG KONG & MACAU' or region == 'SOUTHEAST ASIA EMERGING' or region == 'MAINLAND CHINA':
			return 'G13'
		else:
			return 'others'
	else :
		return 'NA'


udfsrch_region_star_rating=udf(srch_region_star_rating, StringType())
udfsrch_region_city=udf(srch_region_city, StringType())
udfsrch_region_hotel_type=udf(srch_region_Hotel_type, StringType())
udfsrch_region_market=udf(srch_region_market, StringType())
udfsrch_model_name= udf(srch_model_name,StringType())
udfsrch_region_bucket = udf (srch_region_bucket,StringType())


In [3]:
path_hotel_dim = 's3n://ewe-core-meta-prod/CORE/HOTEL_DIM/*'
hotel_dim = sqlCtx.read.format("com.databricks.spark.csv").\
			options(header = False, inferSchema = True, delimiter = "\t", nullValue= '\\N').\
			load(path_hotel_dim)
hotel_dim = hotel_dim.distinct()
hotel_dim = hotel_dim.filter(hotel_dim.C12 == 'BOOKABLE')
hotel_dim = hotel_dim.select("C0","C7","C25","C26","C4","C13","C3")
hotel_dim = hotel_dim.cache()

In [4]:
hotel_dim.count()

246300

In [5]:
hotel_dim = hotel_dim.withColumnRenamed("C0","hotel_id").withColumnRenamed("C7" , "city").withColumnRenamed("C25","region").withColumnRenamed("C26","market").\
			withColumnRenamed("C4","brand").withColumnRenamed("C13","star_rating").withColumnRenamed("C3","hotel_type")

#removing the records with hotel id 0
hotel_dim = hotel_dim.filter((hotel_dim['hotel_id'].isNotNull()))
hotel_dim = hotel_dim.withColumnRenamed('hotel_id','HOTEL_ID')
hotel_dim = hotel_dim.cache()


In [6]:
hotel_dim.count()

246300

In [7]:
hotel_dim_model_name = hotel_dim.withColumn("MODEL_NAME", udfsrch_model_name("region"))
hotel_dim_bkts = hotel_dim_model_name.withColumn("STAR_RATING_BUCKET", udfsrch_region_star_rating("star_rating")).\
                    withColumn("HOTEL_TYPE_BUCKET", udfsrch_region_hotel_type("hotel_type","MODEL_NAME")).\
                    withColumn("MARKET_BUCKET",udfsrch_region_market("market","MODEL_NAME")).\
                    withColumn("REGION_BUCKET", udfsrch_region_bucket("region"))\
                    .withColumn("CITY_BUCKET", udfsrch_region_city("city","MODEL_NAME"))
					
hotel_dim_final = hotel_dim_bkts.filter(hotel_dim_bkts.MODEL_NAME!='NOT APPLICABLE').drop('city').drop('market').drop('brand').drop('star_rating').drop('hotel_type')

hotel_dim_final = hotel_dim_final.cache()
hotel_dim.unpersist()


DataFrame[HOTEL_ID: string, city: string, region: string, market: string, brand: string, star_rating: string, hotel_type: string]

In [8]:
hotel_dim_final.count()

207522

In [9]:
def data_type_change_int(df,cols):
    temp = df
    for col in cols:
        temp = temp.withColumn(col+'_1',df[col].cast(IntegerType())).drop(col).withColumnRenamed(col+'_1',col)
    return temp

def data_type_change_float(df,cols):
    temp = df
    for col in cols:
        temp = temp.withColumn(col+'_1',df[col].cast(DoubleType())).drop(col).withColumnRenamed(col+'_1',col)
    return temp

def searchToCategory(srch_los,):
    if srch_los == 0:
        return '0'
    elif srch_los == 1:
        return '1'
    elif srch_los == 2:
        return '2'
    elif srch_los == 3:
        return '3'
    elif srch_los == 4:
        return '4'
    elif srch_los == 5:
        return '5'
    elif srch_los == 6:
        return '6'
    elif srch_los == 7:
        return '7'
    elif srch_los >= 8 and srch_los <= 14:
        return '8-14'
    else:
        return '>14'


def searchbucketToCategory(SRCH_WINDOW):
    if SRCH_WINDOW <= 0:
        return '<=0'
    elif SRCH_WINDOW == 1:
        return '1'
    elif SRCH_WINDOW == 2:
        return '2'
    elif SRCH_WINDOW >= 3 and SRCH_WINDOW <= 4:
        return '3-4'
    elif SRCH_WINDOW >= 5 and SRCH_WINDOW <= 7:
        return '5-7'
    elif SRCH_WINDOW >= 8 and SRCH_WINDOW <= 14:
        return '8-14'
    elif SRCH_WINDOW >= 15 and SRCH_WINDOW <= 20:
        return '15-20'
    elif SRCH_WINDOW >= 21 and SRCH_WINDOW <= 30:
        return '21-30'
    elif SRCH_WINDOW >= 31 and SRCH_WINDOW <= 40:
        return '31-40'
    elif SRCH_WINDOW >= 41 and SRCH_WINDOW <= 60:
        return '41-60'
    elif SRCH_WINDOW >= 61 and SRCH_WINDOW <= 90:
        return '61-90'
    elif SRCH_WINDOW >= 91 and SRCH_WINDOW <= 120:
        return '91-120'
    elif SRCH_WINDOW >= 121 and SRCH_WINDOW <= 180:
        return '121-180'
    elif SRCH_WINDOW >= 181 and SRCH_WINDOW <= 240:
        return '181-240'
    else:
        return '>240'


def srchBookedLosBucket(booked_los):
    if booked_los is None:
        return '0'
    elif booked_los == 0:
        return '0'
    elif booked_los == 1:
        return '1'
    elif booked_los == 2:
        return '2'
    elif booked_los == 3:
        return '3'
    elif booked_los == 4:
        return '4'
    elif booked_los == 5:
        return '5'
    elif booked_los == 6:
        return '6'
    elif booked_los == 7:
        return '7'
    elif booked_los >= 8 and booked_los <= 14:
        return '8-14'
    else:
        return '>14'


def srchBookedBwBucket(BOOKED_BKG_WINDOW):
    if BOOKED_BKG_WINDOW is None:
        return '0'
    elif BOOKED_BKG_WINDOW <= 0:
        return '<= 0'
    elif BOOKED_BKG_WINDOW == 1:
        return '1'
    elif BOOKED_BKG_WINDOW == 2:
        return '2'
    elif BOOKED_BKG_WINDOW >= 3 or BOOKED_BKG_WINDOW <= 4:
        return '3-4'
    elif BOOKED_BKG_WINDOW >= 5 or BOOKED_BKG_WINDOW <= 7:
        return '5-7'
    elif BOOKED_BKG_WINDOW >= 8 or BOOKED_BKG_WINDOW <= 14:
        return '8-14'
    elif BOOKED_BKG_WINDOW >= 15 or BOOKED_BKG_WINDOW <= 20:
        return '15-20'
    elif BOOKED_BKG_WINDOW >= 21 or BOOKED_BKG_WINDOW <= 30:
        return '21-30'
    elif BOOKED_BKG_WINDOW >= 31 or BOOKED_BKG_WINDOW <= 40:
        return '31-40'
    elif BOOKED_BKG_WINDOW >= 41 or BOOKED_BKG_WINDOW <= 60:
        return '41-60'
    elif BOOKED_BKG_WINDOW >= 61 or BOOKED_BKG_WINDOW <= 90:
        return '61-90'
    elif BOOKED_BKG_WINDOW >= 91 or BOOKED_BKG_WINDOW <= 120:
        return '91-120'
    elif BOOKED_BKG_WINDOW >= 121 or BOOKED_BKG_WINDOW <= 180:
        return '121-180'
    elif BOOKED_BKG_WINDOW >= 181 or BOOKED_BKG_WINDOW <= 240:
        return '181-240'
    else:
        return '>240'


def booking_flag(booked_flag):
    if booked_flag == 'Y':
        return 1
    else:
        return 0
		
# Registering Functions as UDF's

udfsearchToCategory=udf(searchToCategory, StringType())
udfsearchbucketToCategory=udf(searchbucketToCategory, StringType())
udfsrchBookedLosBucket=udf(srchBookedLosBucket, StringType())
udfsrchsrchBookedBwBucket=udf(srchBookedBwBucket, StringType())
booking_flag_udf=udf(booking_flag, IntegerType())


In [10]:
from datetime import *
import time
start_date = datetime.strptime(time.strftime("%Y-%m-%d"), "%Y-%m-%d") #STARTING DATE
no_days = 29  #NO. OF DAYS FOR WHICH DATA IS REQUIRED
date_list = []
for i in range(no_days):
    x = (start_date - timedelta(i))
    x = x.strftime("%Y-%m-%d")
    date_list.append(x)
	
start_date_last = start_date.replace(year=start_date.year - 1)
for i in range(1,8):
    x = (start_date_last - timedelta(i))  ##need to change this while giving for final recommendations
    x = x.strftime("%Y-%m-%d")
    date_list.append(x)
	
##SCHEMA FOR hcba TABLE
hcba_schema = StructType([	StructField('PARTNER_POS',StringType(), True),
							StructField('DEVICE_TYPE',StringType(), True),
							StructField('CLICKED_HOTEL_ID',IntegerType(), True),
							StructField('SRCH_WINDOW',IntegerType(), True),
							StructField('SRCH_LOS',IntegerType(), True),
							StructField('TRAFFIC',IntegerType(), True),
							StructField('COST',DoubleType(), True),
							StructField('BOOKED_FLAG',StringType(), True),
							StructField('BOOKED_HOTEL_ID',IntegerType(), True),
							StructField('BOOKED_LOS',IntegerType(), True),
							StructField('BOOKED_BKG_WINDOW',IntegerType(), True),
							StructField('GROSS_PROFIT',DoubleType(), True),
							StructField('PARTNER_ORG',StringType(), True),
							StructField('ACTIVITY_DATE',StringType(), True)
						])
 

##CREATE EMPTY DATAFRAME 
hcba = sqlContext.createDataFrame(sc.emptyRDD(), hcba_schema)

##APPENDING THE DATA IN ABOVE CREATED hcba TABLE 
for dates in date_list:
	path_hcba = 's3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date='+dates+'/*'  
	print (path_hcba)
	try :
		temp_hcba = sqlCtx.read.format("com.databricks.spark.csv").options(header = False, inferschema = True, delimiter = "\t",nullValue= '\\N').load(path_hcba)
		print('file exist')
		temp_hcba = temp_hcba.select('C1','C3','C7','C8','C9','C13','C14','C19','C20','C21','C22','C45','C65')
		temp_hcba = temp_hcba.filter((temp_hcba.C65 == 'TRIPADVISOR') & (temp_hcba.C1 == 'CA') & (temp_hcba.C7 != 0) & (temp_hcba.C13 != 0) ) 
		temp_hcba = temp_hcba.withColumn('ACTIVITY_DATE',lit(dates))
		hcba = hcba.unionAll(temp_hcba)
	except: print('file does not exist')

hcba = hcba.cache()


s3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date=2016-06-07/*
file does not exist
s3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date=2016-06-06/*
file does not exist
s3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date=2016-06-05/*
file exist
s3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date=2016-06-04/*
file exist
s3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date=2016-06-03/*
file does not exist
s3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date=2016-06-02/*
file exist
s3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date=2016-06-01/*
file exist
s3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date=2016-05-31/*
file exist
s3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date=2016-05-30/*
file exist
s3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date=2016-05-29/*
file exist
s3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date=2016-05-28/*
file exist
s3n://ewe-core-meta-prod/CO

In [11]:
hcba.count()

313491

In [12]:
cols_int = ['SRCH_LOS','SRCH_WINDOW','TRAFFIC','BOOKED_BKG_WINDOW','BOOKED_LOS']
cols_float = ['COST','GROSS_PROFIT']

hcba = data_type_change_int(hcba,cols_int)
hcba = data_type_change_float(hcba,cols_float)



In [13]:
hcba = hcba.withColumn("SRCH_LOS_BUCKET", udfsearchToCategory("SRCH_LOS")).withColumn("SRCH_BW_BUCKET", udfsearchbucketToCategory("SRCH_WINDOW")).withColumn("BOOKED_LOS_BUCKET", udfsrchBookedLosBucket("BOOKED_LOS")).withColumn("BOOKED_BW_BUCKET", udfsrchsrchBookedBwBucket("BOOKED_BKG_WINDOW")).withColumn("BOOKING", booking_flag_udf("BOOKED_FLAG"))

hcba_los_bw_negative_conv = hcba.filter((hcba['BOOKED_FLAG'] == 'Y') & ((hcba['SRCH_WINDOW'] < -1) | (hcba['SRCH_LOS'] <= 0)))
hcba_los_bw_negative_conv = hcba_los_bw_negative_conv.withColumnRenamed('BOOKED_BKG_WINDOW', 'BOOKING_WINDOW').withColumnRenamed('BOOKED_BW_BUCKET', 'BW_BUCKET').withColumnRenamed('BOOKED_LOS_BUCKET', 'LOS_BUCKET').withColumnRenamed('BOOKED_HOTEL_ID', 'HOTEL_ID').withColumnRenamed('BOOKED_LOS', 'LOS')
hcba_los_bw_negative_conv=hcba_los_bw_negative_conv.select('PARTNER_POS','DEVICE_TYPE','HOTEL_ID','PARTNER_ORG','ACTIVITY_DATE','TRAFFIC','BOOKING_WINDOW','LOS','COST','GROSS_PROFIT','LOS_BUCKET','BW_BUCKET','BOOKING')

#-------------------------------------Selecting all the non converted rows---------------------#
#Selecting the required data
hcba_non_conv = hcba.filter((hcba['BOOKED_FLAG'] == 'N') & (hcba['SRCH_WINDOW'] >= -1) & (hcba['SRCH_LOS'] > 0) & (hcba['GROSS_PROFIT'] == 0.0) )

#Renaming the columns as required
hcba_non_conv = hcba_non_conv.withColumnRenamed('CLICKED_HOTEL_ID', 'HOTEL_ID').withColumnRenamed('SRCH_WINDOW', 'BOOKING_WINDOW').withColumnRenamed('SRCH_BW_BUCKET', 'BW_BUCKET').withColumnRenamed('SRCH_LOS', 'LOS').withColumnRenamed('SRCH_LOS_BUCKET', 'LOS_BUCKET')
hcba_non_conv=hcba_non_conv.select('PARTNER_POS','DEVICE_TYPE','HOTEL_ID','PARTNER_ORG','ACTIVITY_DATE','TRAFFIC','BOOKING_WINDOW','LOS','COST','GROSS_PROFIT','LOS_BUCKET','BW_BUCKET','BOOKING')

#----------------------------------Selecting all converted rows-------------------------------------#
#Selecting the required data
hcba_conv = hcba.filter((hcba['BOOKED_FLAG'] == 'Y') & (hcba['SRCH_WINDOW'] >= -1) & (hcba['SRCH_LOS'] > 0))

#Renaming the columns as required
hcba_conv = hcba_conv.withColumnRenamed('CLICKED_HOTEL_ID', 'HOTEL_ID').withColumnRenamed('SRCH_WINDOW', 'BOOKING_WINDOW').withColumnRenamed('SRCH_BW_BUCKET', 'BW_BUCKET').withColumnRenamed('SRCH_LOS', 'LOS').withColumnRenamed('SRCH_LOS_BUCKET', 'LOS_BUCKET')
hcba_conv=hcba_conv.select('PARTNER_POS','DEVICE_TYPE','HOTEL_ID','PARTNER_ORG','ACTIVITY_DATE','TRAFFIC','BOOKING_WINDOW','LOS','COST','GROSS_PROFIT','LOS_BUCKET','BW_BUCKET','BOOKING')


#---Union of all the above 3 tables hcba_non_conv,hcba_conv,hcba_los_bw_negative_conv----------#
hcba_union = hcba_los_bw_negative_conv.unionAll(hcba_non_conv).unionAll(hcba_conv)
hcba_union = hcba_union.cache()
hcba.unpersist()


DataFrame[PARTNER_POS: string, DEVICE_TYPE: string, CLICKED_HOTEL_ID: string, BOOKED_FLAG: string, BOOKED_HOTEL_ID: string, PARTNER_ORG: string, ACTIVITY_DATE: string, SRCH_LOS: int, SRCH_WINDOW: int, TRAFFIC: int, BOOKED_BKG_WINDOW: int, BOOKED_LOS: int, COST: double, GROSS_PROFIT: double, SRCH_LOS_BUCKET: string, SRCH_BW_BUCKET: string, BOOKED_LOS_BUCKET: string, BOOKED_BW_BUCKET: string, BOOKING: int]

In [14]:
hcba_union.count()

311387

In [15]:
opti_strt_date = start_date.strftime("%Y-%m-%d")

end_date = (start_date - timedelta(15))
opti_end_date = end_date.strftime("%Y-%m-%d")

hcba_opti_union = hcba_union.filter((hcba_union.ACTIVITY_DATE <= opti_strt_date) & (hcba_union.ACTIVITY_DATE >=opti_end_date))

##populating week column
hcba_opti_union = hcba_opti_union.withColumn('WEEK',weekofyear(hcba_opti_union.ACTIVITY_DATE))


##populating average weekly traffic
import pyspark.sql.functions as func
hcba_opti_union_agg = hcba_opti_union.groupBy('HOTEL_ID','LOS_BUCKET','BW_BUCKET')
hcba_opti_union_agg1 = hcba_opti_union_agg.agg({'TRAFFIC':'sum'}).withColumnRenamed('sum(TRAFFIC)','TOTAL_TRAFFIC')
hcba_opti_union_agg2 = hcba_opti_union_agg.agg(func.countDistinct('WEEK')).withColumnRenamed('count(WEEK)','NO_WEEKS')

hcba_opti_union = hcba_opti_union.join(hcba_opti_union_agg1,['HOTEL_ID','LOS_BUCKET','BW_BUCKET'],'inner')
hcba_opti_union = hcba_opti_union.join(hcba_opti_union_agg2,['HOTEL_ID','LOS_BUCKET','BW_BUCKET'],'inner')

hcba_opti_union = hcba_opti_union.withColumn('AVERAGE_WEEKLY_TRAFFIC',hcba_opti_union.TOTAL_TRAFFIC/hcba_opti_union.NO_WEEKS)
hcba_opti_union = hcba_opti_union.select('HOTEL_ID','LOS_BUCKET','BW_BUCKET','AVERAGE_WEEKLY_TRAFFIC','DEVICE_TYPE')
hcba_opti_union = hcba_opti_union.cache()


In [16]:
hcba_opti_union.count()

91409

In [17]:
hcba_rf_union = hcba_union[~((hcba_union.ACTIVITY_DATE <= opti_strt_date) & (hcba_union.ACTIVITY_DATE >=opti_end_date))]
hcba_rf_union = hcba_rf_union.cache()

hcba_union.unpersist()


DataFrame[PARTNER_POS: string, DEVICE_TYPE: string, HOTEL_ID: string, PARTNER_ORG: string, ACTIVITY_DATE: string, TRAFFIC: int, BOOKING_WINDOW: int, LOS: int, COST: double, GROSS_PROFIT: double, LOS_BUCKET: string, BW_BUCKET: string, BOOKING: int]

In [18]:
hcba_rf_union.count()

219978

In [19]:
def seller_rank_bucket(seller_rank):
	seller_rank = int(seller_rank)
	if seller_rank <=3 :
		return str(seller_rank)
	else:
		return ">=4"

seller_rank_bucket_udf = udf(seller_rank_bucket,StringType())



##CREATE EMPTY DATAFRAME
mbap_schema = StructType([StructField('ACTIVITY_DATE',StringType(),True),
                          StructField('DEVICE_TYPE',StringType(),True),
                          StructField('HOTEL_ID',IntegerType(),True),
                          StructField('CPC',DoubleType(),True),
                          StructField('SELLER_RANK',DoubleType(),True)
                         ])
mbap = sqlContext.createDataFrame(sc.emptyRDD(),mbap_schema)


In [20]:
##APPENDING THE DATA IN ABOVE CREATED HCBA TABLE 
from datetime import *
import time
start_date = datetime.strptime(time.strftime("%Y-%m-%d"), "%Y-%m-%d") #STARTING DATE
no_days = 15  #NO. OF DAYS FOR WHICH DATA IS REQUIRED
date_list_mbap = []
for i in range(no_days):
    x = (start_date - timedelta(i))
    x = x.strftime("%Y-%m-%d")
    date_list_mbap.append(x)
	
start_date_last = start_date.replace(year=start_date.year - 1)
for i in range(1,8):
    x = (start_date_last - timedelta(i))  ##need to change this while giving for final recommendations
    x = x.strftime("%Y-%m-%d")
    date_list_mbap.append(x)

for dates in date_list_mbap:
	path_mbap =  's3n://ewe-meta-prod/EXPEDIA/ARCHIVE/bidding/TRIPADVISOR_mBAP_bidoutput_'+dates.replace("-","")+'.tab.gz'   
	print (path_mbap)
	try :
		temp_mbap = sqlCtx.read.format("com.databricks.spark.csv").options(header = True, inferschema = True, delimiter = "\t",nullValue= '\\N').load(path_mbap)
		print('file exist')
		temp_mbap = temp_mbap.filter(temp_mbap.posa == 'CA')
		temp_mbap = temp_mbap.select('snapshot_day','device','hotelid','final_bid_value_cpc','expected_rank')
		mbap = mbap.unionAll(temp_mbap)
	except: print('file does not exist')

mbap = mbap.cache()



s3n://ewe-meta-prod/EXPEDIA/ARCHIVE/bidding/TRIPADVISOR_mBAP_bidoutput_20160607.tab.gz
file does not exist
s3n://ewe-meta-prod/EXPEDIA/ARCHIVE/bidding/TRIPADVISOR_mBAP_bidoutput_20160606.tab.gz
file exist
s3n://ewe-meta-prod/EXPEDIA/ARCHIVE/bidding/TRIPADVISOR_mBAP_bidoutput_20160605.tab.gz
file exist
s3n://ewe-meta-prod/EXPEDIA/ARCHIVE/bidding/TRIPADVISOR_mBAP_bidoutput_20160604.tab.gz
file exist
s3n://ewe-meta-prod/EXPEDIA/ARCHIVE/bidding/TRIPADVISOR_mBAP_bidoutput_20160603.tab.gz
file exist
s3n://ewe-meta-prod/EXPEDIA/ARCHIVE/bidding/TRIPADVISOR_mBAP_bidoutput_20160602.tab.gz
file exist
s3n://ewe-meta-prod/EXPEDIA/ARCHIVE/bidding/TRIPADVISOR_mBAP_bidoutput_20160601.tab.gz
file exist
s3n://ewe-meta-prod/EXPEDIA/ARCHIVE/bidding/TRIPADVISOR_mBAP_bidoutput_20160531.tab.gz
file exist
s3n://ewe-meta-prod/EXPEDIA/ARCHIVE/bidding/TRIPADVISOR_mBAP_bidoutput_20160530.tab.gz
file exist
s3n://ewe-meta-prod/EXPEDIA/ARCHIVE/bidding/TRIPADVISOR_mBAP_bidoutput_20160529.tab.gz
file exist
s3n://ewe-m

In [21]:
mbap.count()

8615429

In [22]:
mbap_opti = mbap.filter((mbap.ACTIVITY_DATE <= opti_strt_date) & (mbap.ACTIVITY_DATE >= opti_end_date))
#mbap_opti = mbap
mbap1 = mbap_opti.groupBy('HOTEL_ID','DEVICE_TYPE').agg({"SELLER_RANK":"avg","CPC":"avg"}).withColumnRenamed("avg(SELLER_RANK)","SELLER_RANK").\
		withColumnRenamed("avg(CPC)","CPC")
mbap1_opti = mbap1.withColumn('SELLER_RANK_BUCKET',seller_rank_bucket_udf(mbap1.SELLER_RANK)).select('HOTEL_ID','DEVICE_TYPE','SELLER_RANK','CPC','SELLER_RANK_BUCKET')
mbap1_opti = mbap1_opti.cache()


In [23]:
mbap1_opti.count()

619880

In [24]:
mbap_rf = mbap
#mbap_rf = mbap
mbap_rf = mbap_rf.cache()

mbap.unpersist()


DataFrame[ACTIVITY_DATE: string, DEVICE_TYPE: string, HOTEL_ID: string, CPC: string, SELLER_RANK: string]

In [25]:
mbap_rf.count()

8615429

In [26]:
tripadvisor_ad_final = hcba_rf_union.join(hotel_dim_final,"HOTEL_ID","left_outer")
tripadvisor_ad_final = tripadvisor_ad_final.cache()

In [27]:
tripadvisor_ad_final.count()

219978

In [28]:
hcba_rf_union.unpersist()

tripadvisor_ad_final1 = tripadvisor_ad_final.join(mbap_rf,['ACTIVITY_DATE','HOTEL_ID','DEVICE_TYPE'],'left_outer')
tripadvisor_ad_final1 = tripadvisor_ad_final1.cache()


In [29]:
tripadvisor_ad_final1.count()

219978

In [30]:
tripadvisor_ad_final.unpersist()


DataFrame[HOTEL_ID: string, PARTNER_POS: string, DEVICE_TYPE: string, PARTNER_ORG: string, ACTIVITY_DATE: string, TRAFFIC: int, BOOKING_WINDOW: int, LOS: int, COST: double, GROSS_PROFIT: double, LOS_BUCKET: string, BW_BUCKET: string, BOOKING: int, region: string, MODEL_NAME: string, STAR_RATING_BUCKET: string, HOTEL_TYPE_BUCKET: string, MARKET_BUCKET: string, REGION_BUCKET: string, CITY_BUCKET: string]

In [31]:
seller_rank_look_up = mbap_rf[mbap_rf.SELLER_RANK.isNotNull()]
seller_rank_look_up = seller_rank_look_up.groupBy('HOTEL_ID','SELLER_RANK').count()
seller_rank_look_up = seller_rank_look_up.withColumn("Rank",rank().over(Window.partitionBy('HOTEL_ID').orderBy(seller_rank_look_up["count"].desc())))
seller_rank_look_up = seller_rank_look_up.filter(seller_rank_look_up.Rank == 1).drop('Rank').drop('count').withColumnRenamed('SELLER_RANK','Rank_Freq')

seller_rank_look_up = seller_rank_look_up.cache()


In [32]:
seller_rank_look_up.count()

340439

In [33]:
tripadvisor_ad_final2 = tripadvisor_ad_final1.join(seller_rank_look_up,'HOTEL_ID','inner')
tripadvisor_ad_final3 = tripadvisor_ad_final2.withColumnRenamed('SELLER_RANK','seller_rank_0').withColumnRenamed('ACTIVITY_DATE','ACTIVITY_DATE_ACT')

tripadvisor_ad_final3 = tripadvisor_ad_final3.cache()


In [34]:
tripadvisor_ad_final3.count()

234604

In [35]:
tripadvisor_ad_final1.unpersist()


DataFrame[ACTIVITY_DATE: string, HOTEL_ID: string, DEVICE_TYPE: string, PARTNER_POS: string, PARTNER_ORG: string, TRAFFIC: int, BOOKING_WINDOW: int, LOS: int, COST: double, GROSS_PROFIT: double, LOS_BUCKET: string, BW_BUCKET: string, BOOKING: int, region: string, MODEL_NAME: string, STAR_RATING_BUCKET: string, HOTEL_TYPE_BUCKET: string, MARKET_BUCKET: string, REGION_BUCKET: string, CITY_BUCKET: string, CPC: string, SELLER_RANK: string]

In [36]:
tripadvisor_ad_final4 = tripadvisor_ad_final3.withColumn("SELLER_RANK",coalesce("seller_rank_0","Rank_Freq")).withColumnRenamed('ACTIVITY_DATE_ACT','ACTIVITY_DATE')

In [37]:
tripadvisor_ad_final4 = tripadvisor_ad_final4.drop("seller_rank_0").drop("Rank_Freq")

In [38]:
tripadvisor_ad_final4.cache()

DataFrame[HOTEL_ID: string, ACTIVITY_DATE: string, DEVICE_TYPE: string, PARTNER_POS: string, PARTNER_ORG: string, TRAFFIC: int, BOOKING_WINDOW: int, LOS: int, COST: double, GROSS_PROFIT: double, LOS_BUCKET: string, BW_BUCKET: string, BOOKING: int, region: string, MODEL_NAME: string, STAR_RATING_BUCKET: string, HOTEL_TYPE_BUCKET: string, MARKET_BUCKET: string, REGION_BUCKET: string, CITY_BUCKET: string, CPC: string, SELLER_RANK: string]

In [39]:
tripadvisor_ad_final4.count()

234604

In [40]:
tripadvisor_ad_final3.unpersist()

DataFrame[HOTEL_ID: string, ACTIVITY_DATE_ACT: string, DEVICE_TYPE: string, PARTNER_POS: string, PARTNER_ORG: string, TRAFFIC: int, BOOKING_WINDOW: int, LOS: int, COST: double, GROSS_PROFIT: double, LOS_BUCKET: string, BW_BUCKET: string, BOOKING: int, region: string, MODEL_NAME: string, STAR_RATING_BUCKET: string, HOTEL_TYPE_BUCKET: string, MARKET_BUCKET: string, REGION_BUCKET: string, CITY_BUCKET: string, CPC: string, seller_rank_0: string, Rank_Freq: string]

In [41]:
tripadvisor_ad_final5 = data_type_change_int(tripadvisor_ad_final4,["SELLER_RANK"])
tripadvisor_ad_final_rf = tripadvisor_ad_final5.withColumn('SELLER_RANK_BUCKET',seller_rank_bucket_udf("SELLER_RANK"))
tripadvisor_ad_final_rf.cache()

DataFrame[HOTEL_ID: string, ACTIVITY_DATE: string, DEVICE_TYPE: string, PARTNER_POS: string, PARTNER_ORG: string, TRAFFIC: int, BOOKING_WINDOW: int, LOS: int, COST: double, GROSS_PROFIT: double, LOS_BUCKET: string, BW_BUCKET: string, BOOKING: int, region: string, MODEL_NAME: string, STAR_RATING_BUCKET: string, HOTEL_TYPE_BUCKET: string, MARKET_BUCKET: string, REGION_BUCKET: string, CITY_BUCKET: string, CPC: string, SELLER_RANK: int, SELLER_RANK_BUCKET: string]

In [42]:
tripadvisor_ad_final_rf.count()

234604

In [43]:
from datetime import *
import time
start_date = datetime.strptime(time.strftime("%Y-%m-%d"), "%Y-%m-%d") #STARTING DATE

no_days = 2  #NO. OF DAYS FOR WHICH DATA IS REQUIRED
date_list_thpa = []
for i in range(no_days):
    x = (start_date - timedelta(i))
    x = x.strftime("%Y-%m-%d")
    date_list_thpa.append(x)


In [44]:
TOP_schema = StructType([
						   StructField('DEVICE_TYPE',StringType(), True),
						   StructField('HOTEL_ID',IntegerType(), True),
						   StructField('BOOKING_WINDOW',IntegerType(), True),
						   StructField('LOS',IntegerType(), True),
						   StructField('ADULT_COUNT',DoubleType(), True), 
						   StructField('BASE_RATE_MIN',DoubleType(), True),
						   StructField('COMMISION_AVG',DoubleType(), True),
						   StructField('VARIABLE_MARGIN_AVG',DoubleType(), True),
						   StructField('LOCAL_DATE',StringType(), True),
						 
						 ])

thpa = sqlContext.createDataFrame(sc.emptyRDD(), TOP_schema)


In [45]:
for dates in date_list_thpa:
	path_thpa = 's3n://ewe-core-meta-prod/CORE/TOP_HOTEL_PRICE_AGG/local_date='+dates+'/*'  
	print (path_thpa)
	try :
		temp_thpa = sqlCtx.read.format("com.databricks.spark.csv").options(header = False, inferschema = True, delimiter = "\t",nullValue= '\\N').load(path_thpa)
		temp_thpa = temp_thpa.select('C2','C3','C9','C8','C10','C14','C29','C31','C32')  
		print('file exist')
		thpa = thpa.unionAll(temp_thpa)
	except: print('file does not exist')
	
thpa = thpa.distinct()
thpa = data_type_change_float(thpa, ['BASE_RATE_MIN','COMMISION_AVG','VARIABLE_MARGIN_AVG'])
thpa = data_type_change_int(thpa, ['LOS','BOOKING_WINDOW'])
thpa_opti = thpa.cache()


s3n://ewe-core-meta-prod/CORE/TOP_HOTEL_PRICE_AGG/local_date=2016-06-07/*
file does not exist
s3n://ewe-core-meta-prod/CORE/TOP_HOTEL_PRICE_AGG/local_date=2016-06-06/*
file exist


In [46]:
thpa_opti.count()

105502759

In [47]:
def Net_rev(commission_avg, variable_margin_avg):
    x = float(commission_avg) + float(variable_margin_avg)
    return x


In [48]:
Net_revenue_udf = udf(Net_rev, DoubleType())


In [49]:
thpa_opti1 = thpa_opti.withColumn("NET_REVENUE",Net_revenue_udf(thpa_opti.COMMISION_AVG,thpa_opti.VARIABLE_MARGIN_AVG))
thpa_opti2 = thpa_opti1.groupBy("HOTEL_ID","LOS","BOOKING_WINDOW","LOCAL_DATE","BASE_RATE_MIN").agg({'NET_REVENUE':'avg'}).withColumnRenamed('avg(NET_REVENUE)','NET_REVENUE_0')
thpa_opti3  = thpa_opti2.withColumn("LOS_BUCKET", udfsearchToCategory("LOS")).withColumn("BW_BUCKET", udfsearchbucketToCategory("BOOKING_WINDOW"))

thpa_opti3 = thpa_opti3.cache()


In [50]:
thpa_opti3.count()

77665859

In [51]:
thpa_opti.unpersist()


DataFrame[DEVICE_TYPE: string, HOTEL_ID: string, ADULT_COUNT: string, LOCAL_DATE: string, BASE_RATE_MIN: double, COMMISION_AVG: double, VARIABLE_MARGIN_AVG: double, LOS: int, BOOKING_WINDOW: int]

In [52]:
los_bucket = ['1','2','3','4','5','6','7','8-14','>14']
bw_bucket =  ['0','1','2','3-4','5-7','8-14','15-20','21-30','31-40','41-60','61-90','91-120','121-180','181-240','>240']

import pandas as pd
los = pd.DataFrame(los_bucket)
bw = pd.DataFrame(bw_bucket)
los_bucket_df = sqlContext.createDataFrame(los).withColumnRenamed('0','LOS_BUCKET')
bw_bucket_df = sqlContext.createDataFrame(bw).withColumnRenamed('0','BW_BUCKET')

bkts = los_bucket_df.join(bw_bucket_df)
bkts = bkts.cache()


In [53]:
bkts.count()

135

In [54]:
hotel_dim_final = hotel_dim_final.repartition(300)

optimization_ad_sq_univ = hotel_dim_final.join(bkts)
optimization_ad_sq_univ = optimization_ad_sq_univ.cache()


In [55]:
optimization_ad_sq_univ.count()

28015470

In [56]:
hotel_dim_final.unpersist()
bkts.unpersist()


DataFrame[LOS_BUCKET: string, BW_BUCKET: string]

In [57]:
optimization_ad_sq_univ_traffic = optimization_ad_sq_univ.join(hcba_opti_union,['HOTEL_ID','LOS_BUCKET','BW_BUCKET'],'left_outer')

optimization_ad_sq_univ_traffic = optimization_ad_sq_univ_traffic.cache()


In [58]:
optimization_ad_sq_univ_traffic.count()

28029799

In [59]:
optimization_ad_sq_univ.unpersist()

optimization_ad_sq_univ_traffic_mbap = optimization_ad_sq_univ_traffic.join(mbap1_opti,['HOTEL_ID','DEVICE_TYPE'],'left_outer')

optimization_ad_sq_univ_traffic_mbap = optimization_ad_sq_univ_traffic_mbap.cache()


In [60]:
optimization_ad_sq_univ_traffic_mbap.count()

28029799

In [61]:
optimization_ad_sq_univ_traffic.unpersist()

optimization_ad = optimization_ad_sq_univ_traffic_mbap.join(thpa_opti3,["HOTEL_ID","LOS_BUCKET","BW_BUCKET"],'left_outer')


optimization_ad = data_type_change_float(optimization_ad,["BASE_RATE_MIN","NET_REVENUE_0"])

optimization_ad = optimization_ad.cache()


In [62]:
optimization_ad.count()

83328752

In [63]:
optimization_ad_sq_univ_traffic_mbap.unpersist()
thpa_opti3.unpersist()


DataFrame[HOTEL_ID: string, LOS: int, BOOKING_WINDOW: int, LOCAL_DATE: string, BASE_RATE_MIN: double, NET_REVENUE_0: double, LOS_BUCKET: string, BW_BUCKET: string]

In [64]:
optimization_ad_Hotel_Agg = optimization_ad.groupBy("HOTEL_ID").agg({'BASE_RATE_MIN':'sum','NET_REVENUE_0':'sum'})
optimization_ad_Hotel_Agg = optimization_ad_Hotel_Agg.withColumn('NET_REVENUE_1',optimization_ad_Hotel_Agg['sum(NET_REVENUE_0)']/optimization_ad_Hotel_Agg['sum(BASE_RATE_MIN)']).\
							drop('sum(NET_REVENUE_0)').drop('sum(BASE_RATE_MIN)')

optimization_ad_Region_Agg = optimization_ad.groupBy("MODEL_NAME").agg({'BASE_RATE_MIN':'sum','NET_REVENUE_0':'sum'})
optimization_ad_Region_Agg = optimization_ad_Region_Agg.withColumn('NET_REVENUE_2',optimization_ad_Region_Agg['sum(NET_REVENUE_0)']/optimization_ad_Region_Agg['sum(BASE_RATE_MIN)']).\
							drop('sum(NET_REVENUE_0)').drop('sum(BASE_RATE_MIN)')

optimization_ad1 = optimization_ad.join(optimization_ad_Hotel_Agg,'HOTEL_ID','inner')



In [65]:
optimization_ad1  = optimization_ad1.repartition(300,"MODEL_NAME")
optimization_ad1.cache()

DataFrame[HOTEL_ID: string, LOS_BUCKET: string, BW_BUCKET: string, DEVICE_TYPE: string, region: string, MODEL_NAME: string, STAR_RATING_BUCKET: string, HOTEL_TYPE_BUCKET: string, MARKET_BUCKET: string, REGION_BUCKET: string, CITY_BUCKET: string, AVERAGE_WEEKLY_TRAFFIC: double, SELLER_RANK: double, CPC: double, SELLER_RANK_BUCKET: string, LOS: int, BOOKING_WINDOW: int, LOCAL_DATE: string, BASE_RATE_MIN: double, NET_REVENUE_0: double, NET_REVENUE_1: double]

In [66]:
optimization_ad1.count()

83328752

In [67]:
optimization_ad_Region_Agg.cache()

DataFrame[MODEL_NAME: string, NET_REVENUE_2: double]

In [68]:
optimization_ad_Region_Agg.count()

6

In [69]:
optimization_ad2 = optimization_ad1.join(optimization_ad_Region_Agg,'MODEL_NAME','inner')

optimization_ad2 = optimization_ad2.cache()

In [70]:
optimization_ad2.count()

83328752

In [71]:
optimization_ad_Region_Agg.unpersist()
optimization_ad1.unpersist()


DataFrame[HOTEL_ID: string, LOS_BUCKET: string, BW_BUCKET: string, DEVICE_TYPE: string, region: string, MODEL_NAME: string, STAR_RATING_BUCKET: string, HOTEL_TYPE_BUCKET: string, MARKET_BUCKET: string, REGION_BUCKET: string, CITY_BUCKET: string, AVERAGE_WEEKLY_TRAFFIC: double, SELLER_RANK: double, CPC: double, SELLER_RANK_BUCKET: string, LOS: int, BOOKING_WINDOW: int, LOCAL_DATE: string, BASE_RATE_MIN: double, NET_REVENUE_0: double, NET_REVENUE_1: double]

In [72]:
optimization_ad3 = optimization_ad2.dropna()

In [73]:
optimization_ad3.cache()

DataFrame[MODEL_NAME: string, HOTEL_ID: string, LOS_BUCKET: string, BW_BUCKET: string, DEVICE_TYPE: string, region: string, STAR_RATING_BUCKET: string, HOTEL_TYPE_BUCKET: string, MARKET_BUCKET: string, REGION_BUCKET: string, CITY_BUCKET: string, AVERAGE_WEEKLY_TRAFFIC: double, SELLER_RANK: double, CPC: double, SELLER_RANK_BUCKET: string, LOS: int, BOOKING_WINDOW: int, LOCAL_DATE: string, BASE_RATE_MIN: double, NET_REVENUE_0: double, NET_REVENUE_1: double, NET_REVENUE_2: double]

In [74]:
optimization_ad3.count()

936151

In [75]:
def net_revenue_missing(net_revenue1,net_revenue2,net_revenue3):
	net_revenue1_temp = int(net_revenue1)
	net_revenue2_temp = int(net_revenue2)
    
	if (net_revenue1_temp == 0) and (net_revenue2_temp != 0):
		return net_revenue2
	elif (net_revenue1_temp == 0) and (net_revenue2_temp == 0):
		return net_revenue3
	else :
		return net_revenue1
		
net_revenue_missing_udf = udf(net_revenue_missing,DoubleType())
optimization_ad_final = optimization_ad3.withColumn('NET_REVENUE',net_revenue_missing_udf('NET_REVENUE_0','NET_REVENUE_1','NET_REVENUE_2'))\
                        .drop('NET_REVENUE_1').drop('NET_REVENUE_0').drop('NET_REVENUE_2')
optimization_ad_final = optimization_ad_final.cache()


In [76]:
optimization_ad_final.count()

936151

In [105]:
optimization_ad_final.toPandas().to_csv("/home/vireddy/optimization_ad_final.csv")

In [106]:
tripadvisor_ad_final_rf.toPandas().to_csv("/home/vireddy/tripadvisor_ad_final_rf.csv")

In [77]:

optimization_ad3.unpersist()



DataFrame[MODEL_NAME: string, HOTEL_ID: string, LOS_BUCKET: string, BW_BUCKET: string, DEVICE_TYPE: string, region: string, STAR_RATING_BUCKET: string, HOTEL_TYPE_BUCKET: string, MARKET_BUCKET: string, REGION_BUCKET: string, CITY_BUCKET: string, AVERAGE_WEEKLY_TRAFFIC: double, SELLER_RANK: double, CPC: double, SELLER_RANK_BUCKET: string, LOS: int, BOOKING_WINDOW: int, LOCAL_DATE: string, BASE_RATE_MIN: double, NET_REVENUE_0: double, NET_REVENUE_1: double, NET_REVENUE_2: double]

In [78]:
import numpy
from pyspark.ml.feature import *
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.mllib.linalg import *
from pyspark.sql.functions import *
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import StringIndexer, VectorIndexer


In [199]:
name = 'Mexico'

if name == 'CANADA':
    region_level = 'CITY_BUCKET'
elif name == 'US':
    region_level = 'region'
elif name in ['Mexico', 'CARIBBEAN', 'North America']:
    region_level = 'MARKET_BUCKET'
else:
    region_level = 'REGION_BUCKET'

training_ad = tripadvisor_ad_final_rf.filter(tripadvisor_ad_final_rf.MODEL_NAME == name).filter(tripadvisor_ad_final_rf.DEVICE_TYPE=='DESKTOP').\
                withColumnRenamed(region_level, 'REGION_LEVEL_BUCKET')

training_ad = training_ad.cache()




In [200]:
training_ad.count()

12603

In [201]:
optimization_test = optimization_ad_final.filter(optimization_ad_final.MODEL_NAME == name).filter(optimization_ad_final.DEVICE_TYPE=='DESKTOP')\
    .withColumnRenamed(region_level, 'REGION_LEVEL_BUCKET')

optimization_test = optimization_test.cache()

In [202]:
optimization_test.count()

38005

In [203]:

# for training the data
# Aggregating the data to the required level
training_ad_T = training_ad.groupBy("BW_BUCKET", "LOS_BUCKET", "HOTEL_TYPE_BUCKET", "STAR_RATING_BUCKET", "REGION_LEVEL_BUCKET", \
                                    "SELLER_RANK_BUCKET").sum("TRAFFIC", "BOOKING")

training_ad_T1 = training_ad_T.withColumnRenamed('sum(TRAFFIC)', 'TRAFFIC')
training_ad_T1 = training_ad_T1.withColumnRenamed(
    'sum(BOOKING)', 'BOOKING')

training_ad_T1 = training_ad_T1.cache()
training_ad.unpersist()


DataFrame[HOTEL_ID: string, ACTIVITY_DATE: string, DEVICE_TYPE: string, PARTNER_POS: string, PARTNER_ORG: string, TRAFFIC: int, BOOKING_WINDOW: int, LOS: int, COST: double, GROSS_PROFIT: double, LOS_BUCKET: string, BW_BUCKET: string, BOOKING: int, region: string, MODEL_NAME: string, STAR_RATING_BUCKET: string, HOTEL_TYPE_BUCKET: string, REGION_LEVEL_BUCKET: string, REGION_BUCKET: string, CITY_BUCKET: string, CPC: string, SELLER_RANK: int, SELLER_RANK_BUCKET: string]

In [204]:
training_ad_T1.count()

5997

In [205]:
# Filtering out the TRAFFIC (Outliers) from data
training_ad_T1 = training_ad_T1[
    ~((training_ad_T1.TRAFFIC <= 4) & (training_ad_T1.BOOKING > 0))]
training_ad_T1 = training_ad_T1[
    ~((training_ad_T1.TRAFFIC == 5) & (training_ad_T1.BOOKING == 2))]
training_ad_T1 = training_ad_T1[
    ~((training_ad_T1.TRAFFIC == 6) & (training_ad_T1.BOOKING == 2))]


In [206]:
training_ad_T2 = training_ad_T1.withColumn("chk_data",lit("train")).withColumn('MODEL_NAME',lit("NA")).withColumn("region",lit("NA"))\
                .withColumn('HOTEL_ID',lit("NA")).withColumn('DEVICE_TYPE',lit("NA")).withColumn('AVERAGE_WEEKLY_TRAFFIC',lit(0))\
                .withColumn('SELLER_RANK',lit(0)).withColumn('CPC',lit(0.0)).withColumn('LOS',lit(0)).withColumn('BOOKING_WINDOW',lit(0))\
                .withColumn('LOCAL_DATE',lit("NA")).withColumn('BASE_RATE_MIN',lit(0.00)).withColumn('NET_REVENUE',lit(0.0))\
                .withColumn('MARKET_BUCKET',lit("NA")).withColumn('REGION_BUCKET',lit("NA")).withColumn('LOCAL_DATE',lit("NA"))
training_ad_T2= training_ad_T2.select("MODEL_NAME","HOTEL_ID","LOS_BUCKET","BW_BUCKET","DEVICE_TYPE","region","STAR_RATING_BUCKET",\
                                      "HOTEL_TYPE_BUCKET","MARKET_BUCKET","REGION_BUCKET","REGION_LEVEL_BUCKET","AVERAGE_WEEKLY_TRAFFIC",\
                                      "SELLER_RANK","CPC","SELLER_RANK_BUCKET","LOS","BOOKING_WINDOW","LOCAL_DATE","BASE_RATE_MIN",\
                                      "NET_REVENUE","TRAFFIC","BOOKING","chk_data")
training_ad_T2.cache()

DataFrame[MODEL_NAME: string, HOTEL_ID: string, LOS_BUCKET: string, BW_BUCKET: string, DEVICE_TYPE: string, region: string, STAR_RATING_BUCKET: string, HOTEL_TYPE_BUCKET: string, MARKET_BUCKET: string, REGION_BUCKET: string, REGION_LEVEL_BUCKET: string, AVERAGE_WEEKLY_TRAFFIC: int, SELLER_RANK: int, CPC: double, SELLER_RANK_BUCKET: string, LOS: int, BOOKING_WINDOW: int, LOCAL_DATE: string, BASE_RATE_MIN: double, NET_REVENUE: double, TRAFFIC: bigint, BOOKING: bigint, chk_data: string]

In [207]:
optimization_test2 = optimization_test.withColumn("TRAFFIC",lit(0)).withColumn("BOOKING",lit(0)).withColumn("chk_data",lit("test"))

In [208]:
test_train_data = optimization_test2.unionAll(training_ad_T2)
test_train_data.cache()

DataFrame[MODEL_NAME: string, HOTEL_ID: string, LOS_BUCKET: string, BW_BUCKET: string, DEVICE_TYPE: string, region: string, STAR_RATING_BUCKET: string, HOTEL_TYPE_BUCKET: string, REGION_LEVEL_BUCKET: string, REGION_BUCKET: string, CITY_BUCKET: string, AVERAGE_WEEKLY_TRAFFIC: double, SELLER_RANK: double, CPC: double, SELLER_RANK_BUCKET: string, LOS: int, BOOKING_WINDOW: int, LOCAL_DATE: string, BASE_RATE_MIN: double, NET_REVENUE: double, TRAFFIC: bigint, BOOKING: bigint, chk_data: string]

In [209]:
test_train_data.count()

43934

In [210]:
# Converting the data type of required columns to factor
def indexStringColumns(df, cols):
    tempdf = df
    for col in cols:
        stringIndexer = StringIndexer(inputCol=col, outputCol=col + "-num")
        si_model = stringIndexer.fit(tempdf)
        tempdf = si_model.transform(tempdf)
    return tempdf

cols = {"BW_BUCKET", "LOS_BUCKET", "HOTEL_TYPE_BUCKET",
        "STAR_RATING_BUCKET", "REGION_LEVEL_BUCKET", "SELLER_RANK_BUCKET"}
dfnumeric = indexStringColumns(test_train_data, cols)
dfnumeric = dfnumeric.cache()
training_ad_T2.unpersist()


DataFrame[MODEL_NAME: string, HOTEL_ID: string, LOS_BUCKET: string, BW_BUCKET: string, DEVICE_TYPE: string, region: string, STAR_RATING_BUCKET: string, HOTEL_TYPE_BUCKET: string, MARKET_BUCKET: string, REGION_BUCKET: string, REGION_LEVEL_BUCKET: string, AVERAGE_WEEKLY_TRAFFIC: int, SELLER_RANK: int, CPC: double, SELLER_RANK_BUCKET: string, LOS: int, BOOKING_WINDOW: int, LOCAL_DATE: string, BASE_RATE_MIN: double, NET_REVENUE: double, TRAFFIC: bigint, BOOKING: bigint, chk_data: string]

In [211]:
dfnumeric.count()

43934

In [212]:

def oneHotEncodeColumns(df, cols):
    tempdf = df
    for col in cols:
        onehotenc = OneHotEncoder(
            inputCol=col + "-num", outputCol=col + "-onehot")
        tempdf = onehotenc.transform(tempdf).drop(col + "-num")
    return tempdf
test_train_data_class = oneHotEncodeColumns(dfnumeric, {
                                           "BW_BUCKET", "LOS_BUCKET", "HOTEL_TYPE_BUCKET", "STAR_RATING_BUCKET", "REGION_LEVEL_BUCKET", "SELLER_RANK_BUCKET"})

test_train_data_class = test_train_data_class.cache()
dfnumeric.unpersist()

DataFrame[MODEL_NAME: string, HOTEL_ID: string, LOS_BUCKET: string, BW_BUCKET: string, DEVICE_TYPE: string, region: string, STAR_RATING_BUCKET: string, HOTEL_TYPE_BUCKET: string, REGION_LEVEL_BUCKET: string, REGION_BUCKET: string, CITY_BUCKET: string, AVERAGE_WEEKLY_TRAFFIC: double, SELLER_RANK: double, CPC: double, SELLER_RANK_BUCKET: string, LOS: int, BOOKING_WINDOW: int, LOCAL_DATE: string, BASE_RATE_MIN: double, NET_REVENUE: double, TRAFFIC: bigint, BOOKING: bigint, chk_data: string, STAR_RATING_BUCKET-num: double, REGION_LEVEL_BUCKET-num: double, BW_BUCKET-num: double, HOTEL_TYPE_BUCKET-num: double, LOS_BUCKET-num: double, SELLER_RANK_BUCKET-num: double]

In [213]:
test_train_data_class.count()

43934

In [214]:
training_ad_T1_Class = test_train_data_class.filter(test_train_data_class.chk_data=="train").drop("chk_data").\
                        select("BW_BUCKET","LOS_BUCKET","HOTEL_TYPE_BUCKET","STAR_RATING_BUCKET","REGION_LEVEL_BUCKET","SELLER_RANK_BUCKET",\
                               "TRAFFIC","BOOKING","STAR_RATING_BUCKET-onehot","REGION_LEVEL_BUCKET-onehot","BW_BUCKET-onehot",\
                               "HOTEL_TYPE_BUCKET-onehot","LOS_BUCKET-onehot","SELLER_RANK_BUCKET-onehot")

training_ad_T1_Class.cache()

DataFrame[BW_BUCKET: string, LOS_BUCKET: string, HOTEL_TYPE_BUCKET: string, STAR_RATING_BUCKET: string, REGION_LEVEL_BUCKET: string, SELLER_RANK_BUCKET: string, TRAFFIC: bigint, BOOKING: bigint, STAR_RATING_BUCKET-onehot: vector, REGION_LEVEL_BUCKET-onehot: vector, BW_BUCKET-onehot: vector, HOTEL_TYPE_BUCKET-onehot: vector, LOS_BUCKET-onehot: vector, SELLER_RANK_BUCKET-onehot: vector]

In [215]:
training_ad_T1_Class.count()

5929

In [216]:
# Transforming the training dataset
# Defining function for the same
def genRows(row):
    TRAFFIC = row[6]
    BOOKING = row[7]
    BOOKINGcounter = 0
    rowList = []

    for i in range(TRAFFIC):
        row1 = row[:]
        # row1[6]=1
        row1 = list(row1)
        row1[6] = 1
        row1 = tuple(row1)
        if BOOKINGcounter < int(BOOKING):
            # row1[7]=1
            row1 = list(row1)
            row1[7] = 1
            row1 = tuple(row1)
        else:
            # row1[7]=0
            row1 = list(row1)
            row1[7] = 0
            row1 = tuple(row1)
        BOOKINGcounter += 1
        rowList.append(row1)
    return rowList

# Applying function on the complete dataset (after converting to rdd)
training_ad_T1_Class_rdd = training_ad_T1_Class.rdd
training_ad_T1_Class_rdd = training_ad_T1_Class_rdd.flatMap(
    lambda x: genRows(x))

training_ad_T1_Class_rdd = training_ad_T1_Class_rdd.cache()    
training_ad_T1_Class.unpersist()

DataFrame[BW_BUCKET: string, LOS_BUCKET: string, HOTEL_TYPE_BUCKET: string, STAR_RATING_BUCKET: string, REGION_LEVEL_BUCKET: string, SELLER_RANK_BUCKET: string, TRAFFIC: bigint, BOOKING: bigint, STAR_RATING_BUCKET-onehot: vector, REGION_LEVEL_BUCKET-onehot: vector, BW_BUCKET-onehot: vector, HOTEL_TYPE_BUCKET-onehot: vector, LOS_BUCKET-onehot: vector, SELLER_RANK_BUCKET-onehot: vector]

In [217]:
training_ad_T1_Class_rdd.count()

18851

In [218]:
# Reconverting to dataframe

from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.mllib.linalg import SparseVector, VectorUDT

schema = StructType([StructField('BW_BUCKET', StringType(), True),
                     StructField('LOS_BUCKET', StringType(), True),
                     StructField('HOTEL_TYPE_BUCKET', StringType(), True),
                     StructField('STAR_RATING_BUCKET', StringType(), True),
                     StructField('REGION_LEVEL_BUCKET',
                                 StringType(), True),
                     StructField('SELLER_RANK_BUCKET', StringType(), True),
                     StructField('TRAFFIC', IntegerType(), True),
                     StructField('BOOKING', IntegerType(), True),
                     StructField('HOTEL_TYPE_BUCKET-onehot',
                                 VectorUDT(), True),
                     StructField('BW_BUCKET-onehot',  VectorUDT(), True),
                     StructField('REGION_LEVEL_BUCKET-onehot',
                                 VectorUDT(), True),
                     StructField('LOS_BUCKET-onehot',  VectorUDT(), True),
                     StructField('SELLER_RANK_BUCKET-onehot',
                                 VectorUDT(), True),
                     StructField('STAR_RATING_BUCKET-onehot',  VectorUDT(), True)])
training_ad_T1_Class_df = sqlContext.createDataFrame(
    training_ad_T1_Class_rdd, schema)

# Converting label to double datatype
toDouble = udf(lambda x: float(x * 1.00), DoubleType())
training_ad_T1_Class_df = training_ad_T1_Class_df.withColumn(
    'BOOKING', toDouble(training_ad_T1_Class_df.BOOKING))

training_ad_T1_Class_df = training_ad_T1_Class_df.cache()
training_ad_T1_Class_rdd.unpersist()


PythonRDD[1772] at RDD at PythonRDD.scala:43

In [219]:
training_ad_T1_Class_df.count()

18851

In [220]:
# Taking only relevant columns as features
colList = training_ad_T1_Class_df.columns
colList.remove('BOOKING')
colList.remove('TRAFFIC')
colList.remove('BW_BUCKET')
colList.remove('LOS_BUCKET')
colList.remove('HOTEL_TYPE_BUCKET')
colList.remove('STAR_RATING_BUCKET')
colList.remove('REGION_LEVEL_BUCKET')
colList.remove('SELLER_RANK_BUCKET')
vecAssembler = VectorAssembler(inputCols=colList, outputCol="features")
train_ad = vecAssembler.transform(training_ad_T1_Class_df).select("BOOKING", "TRAFFIC", "BW_BUCKET", "LOS_BUCKET", "HOTEL_TYPE_BUCKET",
                                                                  "STAR_RATING_BUCKET", "REGION_LEVEL_BUCKET", "SELLER_RANK_BUCKET", "features").withColumnRenamed("BOOKING", "label")


train_ad = train_ad.cache()
training_ad_T1_Class_df.unpersist()



DataFrame[BW_BUCKET: string, LOS_BUCKET: string, HOTEL_TYPE_BUCKET: string, STAR_RATING_BUCKET: string, REGION_LEVEL_BUCKET: string, SELLER_RANK_BUCKET: string, TRAFFIC: int, BOOKING: double, HOTEL_TYPE_BUCKET-onehot: vector, BW_BUCKET-onehot: vector, REGION_LEVEL_BUCKET-onehot: vector, LOS_BUCKET-onehot: vector, SELLER_RANK_BUCKET-onehot: vector, STAR_RATING_BUCKET-onehot: vector]

In [221]:
train_ad.count()

18851

In [222]:
optimization_onehot = test_train_data_class.filter(test_train_data_class.chk_data=="test").drop("chk_data")

In [223]:
col =colList
vecAssembler = VectorAssembler(inputCols=col, outputCol="indexedFeatures")
optimization_df = vecAssembler.transform(optimization_onehot)
optimization_df = optimization_df.cache()
optimization_onehot.unpersist()

DataFrame[MODEL_NAME: string, HOTEL_ID: string, LOS_BUCKET: string, BW_BUCKET: string, DEVICE_TYPE: string, region: string, STAR_RATING_BUCKET: string, HOTEL_TYPE_BUCKET: string, REGION_LEVEL_BUCKET: string, REGION_BUCKET: string, CITY_BUCKET: string, AVERAGE_WEEKLY_TRAFFIC: double, SELLER_RANK: double, CPC: double, SELLER_RANK_BUCKET: string, LOS: int, BOOKING_WINDOW: int, LOCAL_DATE: string, BASE_RATE_MIN: double, NET_REVENUE: double, TRAFFIC: bigint, BOOKING: bigint, STAR_RATING_BUCKET-onehot: vector, REGION_LEVEL_BUCKET-onehot: vector, BW_BUCKET-onehot: vector, HOTEL_TYPE_BUCKET-onehot: vector, LOS_BUCKET-onehot: vector, SELLER_RANK_BUCKET-onehot: vector]

In [224]:
optimization_df1 = optimization_df.select([col for col in optimization_onehot.columns if col not in
                    {"STAR_RATING_BUCKET-onehot","REGION_LEVEL_BUCKET-onehot","BW_BUCKET-onehot",\
                               "HOTEL_TYPE_BUCKET-onehot","LOS_BUCKET-onehot","SELLER_RANK_BUCKET-onehot"}])


In [225]:
optimization_df1.count()

38005

In [226]:
# Random Forest model building
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import StringIndexer, VectorIndexer

labelIndexer = StringIndexer(
    inputCol="label", outputCol="indexedLabel").fit(train_ad)
featureIndexer = VectorIndexer(
    inputCol="features", outputCol="indexedFeatures").fit(train_ad)
rf = RandomForestRegressor(labelCol="indexedLabel", featuresCol="indexedFeatures", featureSubsetStrategy="onethird",
                           numTrees=1000, impurity="variance", maxBins=32)
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])
model = pipeline.fit(train_ad)
RF_Model = model.stages[2]

In [227]:
optimization_df_predict = RF_Model.transform(optimization_df).drop(
    "indexedFeatures").withColumnRenamed("prediction", "CONVERSION_RATE")

optimization_df_predict = optimization_df_predict.cache()

In [228]:
optimization_df_predict.show(4)

+----------+--------+----------+---------+-----------+--------------------+------------------+-----------------+--------------------+-------------+-----------+----------------------+-----------------+------------------+------------------+---+--------------+----------+-------------+-----------+-------+-------+-------------------------+--------------------------+----------------+------------------------+-----------------+-------------------------+--------------------+
|MODEL_NAME|HOTEL_ID|LOS_BUCKET|BW_BUCKET|DEVICE_TYPE|              region|STAR_RATING_BUCKET|HOTEL_TYPE_BUCKET| REGION_LEVEL_BUCKET|REGION_BUCKET|CITY_BUCKET|AVERAGE_WEEKLY_TRAFFIC|      SELLER_RANK|               CPC|SELLER_RANK_BUCKET|LOS|BOOKING_WINDOW|LOCAL_DATE|BASE_RATE_MIN|NET_REVENUE|TRAFFIC|BOOKING|STAR_RATING_BUCKET-onehot|REGION_LEVEL_BUCKET-onehot|BW_BUCKET-onehot|HOTEL_TYPE_BUCKET-onehot|LOS_BUCKET-onehot|SELLER_RANK_BUCKET-onehot|     CONVERSION_RATE|
+----------+--------+----------+---------+-----------+----

# Appending results for all the  regions in one table

In [229]:
optimization_rf = optimization_rf.unionAll(optimization_df_predict)

In [230]:
optimization_rf.cache()

DataFrame[MODEL_NAME: string, HOTEL_ID: string, LOS_BUCKET: string, BW_BUCKET: string, DEVICE_TYPE: string, region: string, STAR_RATING_BUCKET: string, HOTEL_TYPE_BUCKET: string, MARKET_BUCKET: string, REGION_BUCKET: string, REGION_LEVEL_BUCKET: string, AVERAGE_WEEKLY_TRAFFIC: double, SELLER_RANK: double, CPC: double, SELLER_RANK_BUCKET: string, LOS: int, BOOKING_WINDOW: int, LOCAL_DATE: string, BASE_RATE_MIN: double, NET_REVENUE: double, TRAFFIC: bigint, BOOKING: bigint, STAR_RATING_BUCKET-onehot: vector, REGION_LEVEL_BUCKET-onehot: vector, BW_BUCKET-onehot: vector, HOTEL_TYPE_BUCKET-onehot: vector, LOS_BUCKET-onehot: vector, SELLER_RANK_BUCKET-onehot: vector, CONVERSION_RATE: double]

In [231]:
optimization_rf.toPandas().to_csv("/home/vireddy/optimization_rf_canada_mexico.csv")

# Optimization Algorithm

In [232]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql.functions import monotonicallyIncreasingId
import pandas

##Pulp package for Linear Programming Optimization algorithm
import pulp
from pulp import *


In [233]:
def predicted(predicted):
    if predicted < 0.0005:
        predicted = 0.0
    else:
        predicted
    return predicted


##If Net revenue is negative then making it positive 
def net_revenue(net_revenue):
    if net_revenue < 0.0 :
        net_revenue = net_revenue*(-1)
    else:
        net_revenue
    return net_revenue

##Creating the objective function at search query level, combining it while running lp_solve algorithm
def objective_fun(net_revenue,predicted):
    return net_revenue * predicted

##Lower limit for the average weekly Traffic
def lower_limit_fun(net_revenue,average_weekly_traffic):
    if net_revenue >= 100.0:
        limit=0.5*average_weekly_traffic
    elif net_revenue>=50.0 and net_revenue <100.0:
        limit=0.2*average_weekly_traffic
    else:
        limit=0.2*average_weekly_traffic
    return limit 

##creating unique id for each search query
def var_name(id):
    return "T_" + str(id)

##registering the above defined functions as uoptimization_rf
objective_fun_udf = udf(objective_fun, DoubleType())
lower_limit_fun_udf = udf(lower_limit_fun, DoubleType())
var_name_udf= udf(var_name,StringType())
net_revenue_udf = udf(net_revenue, DoubleType())
predicted_udf = udf(predicted , DoubleType())



In [234]:
optimization_t_predict = optimization_rf.withColumn("predicted_new",predicted_udf(optimization_rf.CONVERSION_RATE))
optimization_t_predict = optimization_t_predict.cache()

In [235]:
optimization_t_predict.count()

130151

In [237]:
optimization_rf.unpersist()

##removing the searech queries with 0 Traffic , for such queries we will giving rule base throttle value
optimization_t_Traffic =  optimization_t_predict

##creating new column net_revenue_new by making -ve values of net revenue +ve
optimization_t = optimization_t_Traffic.withColumn("net_revenue_new",net_revenue_udf(optimization_t_Traffic.NET_REVENUE))
optimization_t.cache()

DataFrame[MODEL_NAME: string, HOTEL_ID: string, LOS_BUCKET: string, BW_BUCKET: string, DEVICE_TYPE: string, region: string, STAR_RATING_BUCKET: string, HOTEL_TYPE_BUCKET: string, MARKET_BUCKET: string, REGION_BUCKET: string, REGION_LEVEL_BUCKET: string, AVERAGE_WEEKLY_TRAFFIC: double, SELLER_RANK: double, CPC: double, SELLER_RANK_BUCKET: string, LOS: int, BOOKING_WINDOW: int, LOCAL_DATE: string, BASE_RATE_MIN: double, NET_REVENUE: double, TRAFFIC: bigint, BOOKING: bigint, STAR_RATING_BUCKET-onehot: vector, REGION_LEVEL_BUCKET-onehot: vector, BW_BUCKET-onehot: vector, HOTEL_TYPE_BUCKET-onehot: vector, LOS_BUCKET-onehot: vector, SELLER_RANK_BUCKET-onehot: vector, CONVERSION_RATE: double, predicted_new: double, net_revenue_new: double]

In [238]:
optimization_t.count()

130151

In [239]:
optimization_t_temp = optimization_t.withColumn("objective_fun",objective_fun_udf(optimization_t.net_revenue_new ,optimization_t.predicted_new)).\
                      withColumn("Lower_Limit", lower_limit_fun_udf(optimization_t.net_revenue_new , optimization_t.AVERAGE_WEEKLY_TRAFFIC)).\
					  withColumn("id", monotonicallyIncreasingId())


In [240]:
optimization_t_final = optimization_t_temp.withColumn("var_name" ,var_name_udf(optimization_t_temp.id))

optimization_t_final = optimization_t_final.cache()


In [241]:
optimization_t_final.count()

130151

In [242]:
col_remove = ["STAR_RATING_BUCKET-onehot",
                "REGION_LEVEL_BUCKET-onehot",
                "BW_BUCKET-onehot",
                "HOTEL_TYPE_BUCKET-onehot",
                "LOS_BUCKET-onehot",
                "SELLER_RANK_BUCKET-onehot","LOCAL_DATE"]

optimization_t_final1 = optimization_t_final.select([col for col in optimization_t_final.columns if col not in col_remove ])

In [243]:
optimization_t_final1.cache()


DataFrame[MODEL_NAME: string, HOTEL_ID: string, LOS_BUCKET: string, BW_BUCKET: string, DEVICE_TYPE: string, region: string, STAR_RATING_BUCKET: string, HOTEL_TYPE_BUCKET: string, MARKET_BUCKET: string, REGION_BUCKET: string, REGION_LEVEL_BUCKET: string, AVERAGE_WEEKLY_TRAFFIC: double, SELLER_RANK: double, CPC: double, SELLER_RANK_BUCKET: string, LOS: int, BOOKING_WINDOW: int, BASE_RATE_MIN: double, NET_REVENUE: double, TRAFFIC: bigint, BOOKING: bigint, CONVERSION_RATE: double, predicted_new: double, net_revenue_new: double, objective_fun: double, Lower_Limit: double, id: bigint, var_name: string]

In [244]:
optimization_t_final1.count()

130151

# Optimization code for all the regions in parallel

In [250]:
def part_id(x):
    if x == "CANADA":
        return 1.0
    elif x == "Mexico":
        return 2.0
    elif x == "CARIBBEAN":
        return 3.0
    elif x == "US":
        return 4.0
    elif x == "North America":
        return 5.0
    elif x == "Other Region":
        return 6.0


part_id_udf = udf(part_id,DoubleType())

optimization_t_final_temp = optimization_t_final.withColumn("part_id",part_id_udf(optimization_t_final.MODEL_NAME))
optimization_t_final_temp = optimization_t_final_temp.cache()


In [251]:
optimization_t_final_temp.count()

130151

In [252]:
optimization_t_final_temp.rdd.first()

Row(MODEL_NAME='CANADA', HOTEL_ID='10224910', LOS_BUCKET='2', BW_BUCKET='5-7', DEVICE_TYPE='DESKTOP', region='CANADA', STAR_RATING_BUCKET='2', HOTEL_TYPE_BUCKET='MOTEL', MARKET_BUCKET='NIAGARA FALLS, ON, CAN', REGION_BUCKET='NA', REGION_LEVEL_BUCKET='NIAGARA FALLS', AVERAGE_WEEKLY_TRAFFIC=1.0, SELLER_RANK=3.9285714285714284, CPC=0.3992857142857143, SELLER_RANK_BUCKET='3', LOS=2, BOOKING_WINDOW=5, LOCAL_DATE='2016-06-06', BASE_RATE_MIN=111.83, NET_REVENUE=20.81, TRAFFIC=0, BOOKING=0, STAR_RATING_BUCKET-onehot=SparseVector(7, {3: 1.0}), REGION_LEVEL_BUCKET-onehot=SparseVector(14, {0: 1.0}), BW_BUCKET-onehot=SparseVector(14, {7: 1.0}), HOTEL_TYPE_BUCKET-onehot=SparseVector(6, {2: 1.0}), LOS_BUCKET-onehot=SparseVector(8, {1: 1.0}), SELLER_RANK_BUCKET-onehot=SparseVector(4, {0: 1.0}), CONVERSION_RATE=0.023652251671828606, predicted_new=0.023652251671828606, net_revenue_new=20.81, objective_fun=0.49220335729075326, Lower_Limit=0.2, id=1477468749824, var_name='T_1477468749824', part_id=1.0)

In [253]:
len(optimization_t_final_temp.rdd.first())

36

In [256]:
pairs = optimization_t_final_temp.rdd.map(lambda x : (x[35],x))
pairs_part = pairs.partitionBy(2)

pairs_part = pairs_part.cache()


In [257]:
pairs_part.first()

(2.0,
 Row(MODEL_NAME='Mexico', HOTEL_ID='11547656', LOS_BUCKET='8-14', BW_BUCKET='>240', DEVICE_TYPE='DESKTOP', region='MEXICO - RESORTS REGION', STAR_RATING_BUCKET='4', HOTEL_TYPE_BUCKET='CONDO', MARKET_BUCKET='MEXICO - RIVIERA MAYA, PLAYA DEL CARMEN & TULUM', REGION_BUCKET='NA', REGION_LEVEL_BUCKET='NA', AVERAGE_WEEKLY_TRAFFIC=2.0, SELLER_RANK=2.142857142857143, CPC=0.3607142857142857, SELLER_RANK_BUCKET='2', LOS=13, BOOKING_WINDOW=299, LOCAL_DATE='2016-06-06', BASE_RATE_MIN=1629.28, NET_REVENUE=760.14, TRAFFIC=0, BOOKING=0, STAR_RATING_BUCKET-onehot=SparseVector(7, {1: 1.0}), REGION_LEVEL_BUCKET-onehot=SparseVector(15, {0: 1.0}), BW_BUCKET-onehot=SparseVector(14, {3: 1.0}), HOTEL_TYPE_BUCKET-onehot=SparseVector(6, {3: 1.0}), LOS_BUCKET-onehot=SparseVector(8, {1: 1.0}), SELLER_RANK_BUCKET-onehot=SparseVector(3, {1: 1.0}), CONVERSION_RATE=0.0016377630701102338, predicted_new=0.0016377630701102338, net_revenue_new=760.14, objective_fun=1.244929220113593, Lower_Limit=1.0, id=5214090297

In [261]:
def lp_solve(df):
    import pulp
    from pulp import LpProblem,LpVariable,lpSum,LpStatus,solvers
    df = list(df)
    df_count = len (df)
    region_name = [df[i][1][0] for i in range(df_count) if i == 0]
    t_list = [df[i][1][34] for i in range(df_count)]
    objective_list = [df[i][1][31] for i in range(df_count)]
    final_objective = {t_list[i]: objective_list[i] for i in range(0, df_count)}
    lower_limit_list = [df[i][1][32] for i in range(df_count)]
    upper_limit_list = [df[i][1][11] for i in range(df_count)]
    lower_limit_final = {t_list[i]: lower_limit_list[i] for i in range(0, df_count)}
    upper_limit_final = {t_list[i]: upper_limit_list[i] for i in range(0, df_count)}
    cnt =0.3
    lp_status = 'Start'
    while ((lp_status != "Optimal")  & (cnt <=2.0)):
        a = cnt
        last_row =  [(df[i][1][13] - a*df[i][1][31]) for i in range(df_count)]
        final_constraint = {t_list[i]: last_row[i] for i in range(0, df_count)}
        prob = LpProblem("The Optimization Problem", LpMaximize)
        lp_var = {}
        for variable in t_list:

            variable1 = LpVariable(variable,lower_limit_final[variable],upper_limit_final[variable])
            lp_var[str(variable)] = variable1
        prob += lpSum([final_objective[i]*lp_var[i] for i in t_list]) 
        prob += lpSum([final_constraint[i]*lp_var[i] for i in t_list]) == 0
        prob.solve()
        var_values = {"start":0}
        for v in prob.variables():
            var_values.update({v.name[2:]:v.varValue})
        lp_status = LpStatus[prob.status]
        results={"region_name":region_name,"lp_status":LpStatus[prob.status],"cnt":(cnt), "gp":value(prob.objective),"var_list":var_values}
        cnt = cnt+.1
    yield  results


In [262]:
final_results = pairs_part.mapPartitions(lp_solve).collect()


# Output file generation

In [269]:
def df_create(x):
    mydictionary = x['var_list']
    Efficiency =  x['cnt']
    Model_Name = x['region_name'][0]
    
    var_list = []
    for key in mydictionary:
        var_list.append((key, mydictionary[key],Efficiency,Model_Name))
          
    df = pandas.DataFrame(var_list, columns=['var_id', 'var_value','Efficiency','Model'])
    df_final = sqlContext.createDataFrame(df)
    
    return df_final 



In [270]:
field = [StructField("var_id", StringType(), True),StructField("var_value", DoubleType(), True),StructField("Efficiency", FloatType(), True),StructField("Model", StringType(), True)]
schema = StructType(field)

optimization_df_algo1 = sqlContext.createDataFrame(sc.emptyRDD(), schema)


In [272]:
for i in [0]:
    optimization_df_algo1 = optimization_df_algo1.unionAll(df_create(final_results[i]))


In [274]:
optimization_df_algo1.show(4)

+-------------+---------+----------+------+
|       var_id|var_value|Efficiency| Model|
+-------------+---------+----------+------+
|5214090299349|      3.0|       0.3|Mexico|
|5214090311067|      0.6|       0.3|Mexico|
|5214090301242|      1.0|       0.3|Mexico|
|5214090310043|      0.5|       0.3|Mexico|
+-------------+---------+----------+------+
only showing top 4 rows



In [276]:
optimization_df_algo1 = optimization_df_algo1.cache()


optimization_df_algo1 = optimization_df_algo1.withColumn("id",optimization_df_algo1["var_id"].cast(LongType()))
optimization_df_algo2 = optimization_t_final_temp.select([col for col in optimization_t_final_temp.columns if col not in {"predicted_new","net_revenue_new","objective_fun","Lower_Limit","var_name","part_id"}]).join(optimization_df_algo1,"id","inner")
	
def threshold(conv_rate,avg_weekly_traffic):
    z = conv_rate/avg_weekly_traffic 
    return (1-z)

threshold_udf = udf(threshold,DoubleType())

optimization_df_algo3 = optimization_df_algo2.withColumn("threshold",threshold_udf("var_value","AVERAGE_WEEKLY_TRAFFIC"))

optimization_df_algo3 = optimization_df_algo3.select([col for col in optimization_df_algo3.columns if col not in {"id","var_id","Model"}])
optimization_df_algo3 = optimization_df_algo3.withColumnRenamed("var_value","Lp_solution")
optimization_df_algo_final = optimization_df_algo3.select("HOTEL_ID","LOS_BUCKET","BW_BUCKET","threshold","BASE_RATE_MIN")

optimization_df_algo_final = optimization_df_algo_final.cache()
optimization_df_algo1.unpersist()


DataFrame[var_id: string, var_value: double, Efficiency: double, Model: string, id: bigint]

In [283]:
optimization_df_algo_final.count()

38005

In [284]:
optimization_df_algo_final.show(4)

+--------+----------+---------+---------+-------------+
|HOTEL_ID|LOS_BUCKET|BW_BUCKET|threshold|BASE_RATE_MIN|
+--------+----------+---------+---------+-------------+
| 7474531|         3|    31-40|      0.8|       265.32|
| 6619089|      8-14|  121-180|      0.5|        585.1|
|   10778|         5|   91-120|      0.8|        741.2|
|    8238|         7|  181-240|      0.5|          0.0|
+--------+----------+---------+---------+-------------+
only showing top 4 rows



In [285]:
optimization_throttle_final = optimization_df_algo_final

In [286]:
def minRemainingDays(bwBucket):
     if bwBucket == "<=0":
         return "0"
     elif bwBucket == "1":
         return "1"
     elif bwBucket == "2":
         return "2"
     elif bwBucket == ">240":
         return "240"
     else:
         string_list = bwBucket.split("-")
         return str(string_list[0])
        
def maxRemainingDays(bwBucket):
     if bwBucket == "<=0":
         return "0"
     elif bwBucket == "1":
         return "1"
     elif bwBucket == "2":
         return "2"
     elif bwBucket == ">240":
         return "240"
     else:
         string_list = bwBucket.split("-")
         return str(string_list[1])

minRemainingDays_udf = udf(minRemainingDays,StringType())
maxRemainingDays_udf = udf(maxRemainingDays,StringType())

opti_output = optimization_throttle_final.withColumn("minRemainingDays",minRemainingDays_udf("BW_BUCKET")).\
			  withColumn("maxRemainingDays",maxRemainingDays_udf("BW_BUCKET"))



In [287]:
opti_output.show(4)

+--------+----------+---------+---------+-------------+----------------+----------------+
|HOTEL_ID|LOS_BUCKET|BW_BUCKET|threshold|BASE_RATE_MIN|minRemainingDays|maxRemainingDays|
+--------+----------+---------+---------+-------------+----------------+----------------+
| 7474531|         3|    31-40|      0.8|       265.32|              31|              40|
| 6619089|      8-14|  121-180|      0.5|        585.1|             121|             180|
|   10778|         5|   91-120|      0.8|        741.2|              91|             120|
|    8238|         7|  181-240|      0.5|          0.0|             181|             240|
+--------+----------+---------+---------+-------------+----------------+----------------+
only showing top 4 rows



In [288]:
rule_strt_date = (start_date - timedelta(1))
rule_strt_date = rule_strt_date.strftime("%Y-%m-%d")

rule_end_date = (start_date - timedelta(8))
rule_end_date = rule_end_date.strftime("%Y-%m-%d")

opti_output1 = opti_output.withColumn("ruleStartDate", lit(rule_strt_date))
opti_output2 = opti_output1.withColumn("ruleEndDate", lit(rule_end_date))


# add startDay column 
from_pattern = 'yyyy-mm-dd'
to_pattern = 'EEEEE'
opti_output3 = opti_output2.withColumn('startDay', from_unixtime(unix_timestamp(opti_output2['ruleStartDate'], from_pattern), to_pattern))

opti_output3.cache()
optimization_throttle_final.unpersist()


DataFrame[HOTEL_ID: string, LOS_BUCKET: string, BW_BUCKET: string, threshold: double, BASE_RATE_MIN: double]

In [290]:
opti_output3.count()

38005

In [291]:
opti_output3.show(4)

+--------+----------+---------+---------+-------------+----------------+----------------+-------------+-----------+---------+
|HOTEL_ID|LOS_BUCKET|BW_BUCKET|threshold|BASE_RATE_MIN|minRemainingDays|maxRemainingDays|ruleStartDate|ruleEndDate| startDay|
+--------+----------+---------+---------+-------------+----------------+----------------+-------------+-----------+---------+
| 7474531|         3|    31-40|      0.8|       265.32|              31|              40|   2016-06-06| 2016-05-30|Wednesday|
| 6619089|      8-14|  121-180|      0.5|        585.1|             121|             180|   2016-06-06| 2016-05-30|Wednesday|
|   10778|         5|   91-120|      0.8|        741.2|              91|             120|   2016-06-06| 2016-05-30|Wednesday|
|    8238|         7|  181-240|      0.5|          0.0|             181|             240|   2016-06-06| 2016-05-30|Wednesday|
+--------+----------+---------+---------+-------------+----------------+----------------+-------------+-----------+---

In [292]:
opti_output3_agg = opti_output3.groupBy(["LOS_BUCKET","BW_BUCKET","threshold"]).count()
opti_output3_agg.cache()

opti_output3_temp = opti_output3.groupBy(["LOS_BUCKET","BW_BUCKET","threshold"])


In [295]:
opti_output3_agg.count()

295

In [296]:
import pandas
row_no = list(range(opti_output3_agg.count()))
row_df = pandas.DataFrame(row_no)
row_df = sqlContext.createDataFrame(row_df)
row_df = row_df.withColumn("join",lit(1))

from pyspark.sql.functions import lit

def hotel_grp(id):
    return ("HO"+str(id))
hotel_grp_udf = udf(hotel_grp,StringType())

opti_output3_agg1 = opti_output3_agg.withColumn("join",lit(1))

opti_output3_agg2 = opti_output3_agg1.join(row_df,"join","inner").drop("id").withColumnRenamed("0","id").withColumn("hotel_group",hotel_grp_udf("id")).drop("id").drop("count")
opti_output3_agg2.cache()


DataFrame[join: int, LOS_BUCKET: string, BW_BUCKET: string, threshold: double, hotel_group: string]

In [297]:
opti_output3_agg2.count()

87025

In [298]:
opti_output4 = opti_output3.join(opti_output3_agg2,["LOS_BUCKET","BW_BUCKET","threshold"],"inner").drop("join")
opti_output4.cache()
opti_output3_agg2.unpersist()
opti_output3.unpersist()


DataFrame[HOTEL_ID: string, LOS_BUCKET: string, BW_BUCKET: string, threshold: double, BASE_RATE_MIN: double, minRemainingDays: string, maxRemainingDays: string, ruleStartDate: string, ruleEndDate: string, startDay: string]

In [299]:
opti_output4.count()

11211475

In [300]:
opti_output4.show(4)

+----------+---------+---------+--------+-------------+----------------+----------------+-------------+-----------+---------+-----------+
|LOS_BUCKET|BW_BUCKET|threshold|HOTEL_ID|BASE_RATE_MIN|minRemainingDays|maxRemainingDays|ruleStartDate|ruleEndDate| startDay|hotel_group|
+----------+---------+---------+--------+-------------+----------------+----------------+-------------+-----------+---------+-----------+
|         2|    41-60|      0.0| 1666996|       891.48|              41|              60|   2016-06-06| 2016-05-30|Wednesday|        HO2|
|         2|    41-60|      0.0| 1666996|       669.24|              41|              60|   2016-06-06| 2016-05-30|Wednesday|        HO2|
|         2|    41-60|      0.0| 1666996|       620.76|              41|              60|   2016-06-06| 2016-05-30|Wednesday|        HO2|
|         2|    41-60|      0.0| 1666996|       620.76|              41|              60|   2016-06-06| 2016-05-30|Wednesday|        HO2|
+----------+---------+---------+--

In [303]:
df_los_8 = opti_output4.filter(opti_output4.LOS_BUCKET == '8-14')
df_los_8.cache()

df_los_other = opti_output4.filter(opti_output4.LOS_BUCKET != '8-14')
df_los_other.cache()

opti_output4.cache()

los = list(range(8,15,1))
los_df = pandas.DataFrame(los)
los_df = sqlContext.createDataFrame(los_df)
los_df.cache()

df_los_8 = df_los_8.join(los_df).drop("LOS_BUCKET").withColumnRenamed("0","LOS_BUCKET")

def replace(los):
    if los == '>14':
        return '15'
    else:
        return los

replace_udf = udf(replace,StringType())


In [304]:
df_los_8.cache()

DataFrame[BW_BUCKET: string, threshold: double, HOTEL_ID: string, BASE_RATE_MIN: double, minRemainingDays: string, maxRemainingDays: string, ruleStartDate: string, ruleEndDate: string, startDay: string, hotel_group: string, LOS_BUCKET: bigint]

In [305]:
df_los_8.count()

24160500

In [306]:
def replace(los):
    if los == '>14':
        return '15'
    else:
        return los

replace_udf = udf(replace,StringType())
df_los_all = df_los_8.unionAll(df_los_other)
df_los_8.unpersist()
df_los_other.unpersist()


DataFrame[LOS_BUCKET: string, BW_BUCKET: string, threshold: double, HOTEL_ID: string, BASE_RATE_MIN: double, minRemainingDays: string, maxRemainingDays: string, ruleStartDate: string, ruleEndDate: string, startDay: string, hotel_group: string]

In [307]:
df_los_all.count()

31920475

In [317]:
opti_output5 = df_los_all.withColumn("lengthOfStay",replace_udf("LOS_BUCKET")).withColumn("lengthOfStay",replace_udf("LOS_BUCKET")).drop("LOS_BUCKET")
opti_output5.cache()
df_los_all.unpersist()


## UDF to split AvgBV column
def floor_percentile_bucket(val):
    val = round(val,2)
    if (val>=0.00) & (val<=62.95):
        return 0
    elif (val>=62.96) & (val<=86.32):
        return 62.96
    elif (val>=86.33) & (val<=110.36):
        return 86.33
    elif (val>=110.37) & (val<=142.01):
        return 110.37
    elif (val>=142.02) & (val<=188.92):
        return 142.02
    elif (val>=188.93) & (val<=241.96):
        return 188.93
    elif (val>=241.97) & (val<=324.25):
        return 241.97
    elif (val>=324.26) & (val<=453.64):
        return 324.26
    elif (val>=453.65) & (val<=726.35):
        return 453.65
    elif (val>=726.36) & (val<=7409.94):
        return 726.36
    elif val>=7409.95:
        return 7409.94
    else:
        print("EXCEPTION")
        
def ceiling_percentile_bucket(val):
    val = round(val,2)
    if (val>=0.00) & (val<=62.95):
        return 62.95
    elif (val>=62.96) & (val<=86.32):
        return 86.32
    elif (val>=86.33) & (val<=110.36):
        return 110.36
    elif (val>=110.37) & (val<=142.01):
        return 142.01
    elif (val>=142.02) & (val<=188.92):
        return 188.92
    elif (val>=188.93) & (val<=241.96):
        return 241.96
    elif (val>=241.97) & (val<=324.25):
        return 324.25
    elif (val>=324.26) & (val<=453.64):
        return 453.64
    elif (val>=453.65) & (val<=726.35):
        return 726.35
    elif (val>=726.36) & (val<=7409.94):
        return 7409.94
    elif val>=7409.95:
        return 10000.00
    else:
        return 1
        
udf_floor_percentile_bucket = udf(floor_percentile_bucket,FloatType())
udf_ceiling_percentile_bucket = udf(ceiling_percentile_bucket,FloatType())

opti_output51 = data_type_change_float(opti_output5,["BASE_RATE_MIN"])
opti_output6 = opti_output5.withColumn("floor_percentile",udf_floor_percentile_bucket("BASE_RATE_MIN")).withColumn("ceiling_percentile",udf_ceiling_percentile_bucket("BASE_RATE_MIN"))


In [318]:
opti_output6.cache()

DataFrame[BW_BUCKET: string, threshold: string, HOTEL_ID: string, BASE_RATE_MIN: string, minRemainingDays: string, maxRemainingDays: string, ruleStartDate: string, ruleEndDate: string, startDay: string, hotel_group: string, lengthOfStay: string, floor_percentile: float, ceiling_percentile: float]

# Optimization code for running individually for a region

In [198]:
optimization_t_final1.describe("CONVERSION_RATE","predicted_new").show()

+-------+--------------------+--------------------+
|summary|     CONVERSION_RATE|       predicted_new|
+-------+--------------------+--------------------+
|  count|               92146|               92146|
|   mean| 0.01402962353620312| 0.01402962353620312|
| stddev| 0.00430790871786846| 0.00430790871786846|
|    min|0.005075335592769067|0.005075335592769067|
|    max|0.029639071721393642|0.029639071721393642|
+-------+--------------------+--------------------+



In [167]:
region_df = optimization_t_final1.filter(optimization_t_final1['MODEL_NAME']=='CANADA')
region_df.cache()
region_count = region_df.count() 


In [168]:
region_df.count()

92146

In [183]:
region_df.rdd.first()

Row(MODEL_NAME='CANADA', HOTEL_ID='10224910', LOS_BUCKET='2', BW_BUCKET='5-7', DEVICE_TYPE='DESKTOP', region='CANADA', STAR_RATING_BUCKET='2', HOTEL_TYPE_BUCKET='MOTEL', MARKET_BUCKET='NIAGARA FALLS, ON, CAN', REGION_BUCKET='NA', REGION_LEVEL_BUCKET='NIAGARA FALLS', AVERAGE_WEEKLY_TRAFFIC=1.0, SELLER_RANK=3.9285714285714284, CPC=0.3992857142857143, SELLER_RANK_BUCKET='3', LOS=2, BOOKING_WINDOW=5, BASE_RATE_MIN=111.83, NET_REVENUE=20.81, TRAFFIC=0, BOOKING=0, CONVERSION_RATE=0.023652251671828606, predicted_new=0.023652251671828606, net_revenue_new=20.81, objective_fun=0.49220335729075326, Lower_Limit=0.2, id=1477468749824, var_name='T_1477468749824')

In [196]:
region_df.rdd.first()[24]

0.49220335729075326

In [173]:
t_list = region_df.rdd.map(lambda x  : x[27]).collect()

objective_list =  region_df.rdd.map(lambda x  : x[24]).collect()

final_objective = {t_list[i]: objective_list[i] for i in range(0, region_count)}
                          
lower_limit_list =  region_df.rdd.map(lambda x  : x[25]).collect()
upper_limit_list =  region_df.rdd.map(lambda x  : x[11]).collect()

lower_limit_final = {t_list[i]: lower_limit_list[i] for i in range(0, region_count)}
upper_limit_final = {t_list[i]: upper_limit_list[i] for i in range(0, region_count)}


In [193]:
cnt = 40.0
lp_status = "Start"

while ((lp_status != "Optimal")  & (cnt <=140.0)):
	a = cnt
	def constraint_fun(row):
		last_row = row[13] - a * row[24]
		return last_row
	last_row_RDD = region_df.rdd.map(lambda x: constraint_fun(x))
	last_row = last_row_RDD.collect()
	final_constraint = {t_list[i]: last_row[i] for i in range(0, region_count)}
	prob = LpProblem("The Optimization Problem", LpMaximize)
	lp_var = {}
	for variable in t_list:
		variable = LpVariable(variable,lower_limit_final[variable],upper_limit_final[variable])
		lp_var[str(variable)] = variable
	prob += lpSum([final_objective[i]*lp_var[i] for i in t_list])
	prob += lpSum([final_constraint[i]*lp_var[i] for i in t_list]) == 0
	prob.solve()
	lp_status = LpStatus[prob.status]
	var_values = {"start":0}
	for v in prob.variables():
		var_values.update({v.name[2:]:v.varValue})
	lp_status = LpStatus[prob.status]
	results={"region_name":'CANADA',"lp_status":LpStatus[prob.status],"cnt":(cnt), "gp":value(prob.objective),"var_list":var_values}
	cnt += 40.0
	print(cnt)
    


80.0
120.0
160.0


In [194]:
results

{'cnt': 120.0,
 'gp': -2234483859.651107,
 'lp_status': 'Infeasible',
 'region_name': 'CANADA',
 'var_list': {'1477468808984': 0.2,
  '1477468780877': 2.0,
  '1477468753698': 0.6,
  '1477468773388': 6.0,
  '1477468809153': 0.4,
  '1477468793266': 0.2,
  '1477468761891': 0.5,
  '1477468759592': 0.2,
  '1477468756602': 0.3,
  '1477468841572': 0.6,
  '1477468816704': 0.2,
  '1477468808905': 0.2,
  '1477468819432': 0.2,
  '1477468790601': 0.4,
  '1477468757524': 1.2,
  '1477468774129': 0.4,
  '1477468835813': 0.4,
  '1477468763484': 1.0,
  '1477468780089': 0.5,
  '1477468753650': 0.4,
  '1477468797516': 1.0,
  '1477468826705': 4.0,
  '1477468793642': 0.9,
  '1477468770446': 0.4,
  '1477468751143': 1.2,
  '1477468786264': 0.2,
  '1477468763427': 0.2,
  '1477468838181': 0.2,
  '1477468768861': 0.2,
  '1477468751346': 2.0,
  '1477468802067': 0.4,
  '1477468750972': 0.5,
  '1477468768946': 1.0,
  '1477468757723': 0.4,
  '1477468836001': 0.4,
  '1477468840368': 0.2,
  '1477468809303': 0.2,
  '1